# D-Hydro De Dellen V1

## Introductie

Met dit notebook bouw je een 1D D-hydromodel voor Stroomgebied De Dellen in het beheergebied van waterschap Hunze en Aa's.

Zorg dat je beschikt over een werkende Python installatie en <a href="https://github.com/openearth/delft3dfmpy#installation">D-HYDAMO omgeving</a>. Het resulterende model moet geimporteerd kunnen worden in D-HYDRO versie 0.9.7.52006 of hoger.

De verwijzing naar bestanden is geschreven in het Nederlands. De rest van de code is geschreven in het Engels. Elk code-blok is voorzien van uitleg boven het desbetreffende blok, geschreven in het Nederlands.

Werkwijze:
- Download alle projectbestanden van deze Git en plaats ze in een projectmap op de schijf.
- Download de bestanden met gegevens uit het <a href="https://www.dropbox.com/s/o25okc301vgjmiy/beheerregister.zip?dl=0">beheerregister</a> en zet deze in de folder .\beheerregister binnen het project
- Download de bestanden met <a href="https://www.dropbox.com/s/asbffnnxf5gsar8/data.zip?dl=0">boundaries en laterale instroming</a> en zet deze in de folder .\data binnen het project
- Zorg voor de werkende Delft3DFMPY-omgeving zoals in de aanhef omschreven.
- Activeer de Delft3DFMPY-omgeving als volgt: start een Anaconda Prompt via het startmenu (een console-window)
- Navigeer naar het project met de 'good old' DOS-commando's cd (change directory)
- Voer het commando conda activate delft3dfmpy uit
- Om straks de XLSX-bestanden (boundaries) te kunnen uitlezen is openpyxl nodig: conda install openpyxl. Dit commando moet uitgevoerd worden binnen de delft3dfmpy-environment. Vandaar dat deze stap hier komt.
- Start nu het Jupyter Notebook met het commando Jupyter notebook.
- In de webbrowser verschijnt nu een pagina met een overzicht van de projectbestanden. Klik nu op modelbouw.ipynb om dit notebook te openen.

# Basis

In [44]:
import os
import shutil
import numpy as np
import xml.etree.ElementTree as ET

# Controle bestanden

Alle bestanden die gebruikt worden in deze tutorial staan in het code-blok hieronder. Wanneer je dit codeblok uitvoert wordt de aanwezigheid van deze bestanden gecontroleerd.



In [45]:
from pathlib import Path

data_path = Path(r".\data").absolute().resolve()
excelbestanden = data_path.joinpath("xlsx")
beheerregister = Path(r".\beheerregister").absolute().resolve()
outputdir = Path(r".\dellen").absolute().resolve()
rtc_generic_files = Path(r".\rtc_generic_files").absolute().resolve() #the directory containing RTC generic files to be copied into the project
rtc_results_dir = Path(r".\dellen\rtc").absolute().resolve()

fmdir = os.path.join(outputdir,"fm") #this is the 'original' output dir. It will be replaced later because of RTC coupling

#alle individuele databestanden krijgen een eigen pad
timeseriesfile = excelbestanden.joinpath("timeseries_input.xlsx")

#definieer een naam voor het resultatenbestand
name_mdu = "dellen"

print(rtc_generic_files)

#het pad naar beheerregister bevat de bronbestanden in onveranderde vorm en dus nog hiaten en fouten bevatten
#beheerregister = Path(r".\beheerregister").absolute().resolve()

modelbestanden = {"randvoorwaarden":"randvoorwaarden.xlsx"}

invoerbestanden = {"modelgebied": "DeDellen_gebiedsgrens.shp",
                   "branches": "Hoofdwatergang_Dellen_singlepart.shp",
                   "profielpunten": "profielpunten_Dellen.shp",
                   "bruggen": "Brug_Dellen.shp",
                   "duikers": "Duiker_Dellen.shp",
                   "sifons": "Syphon_Dellen.shp",
                   "stuwen": "Stuw_Dellen.shp",
                   "inlaten": "Inlaat_Dellen.shp",
                   "gemalen": "Gemaal_Dellen.shp",
                   "peilgebieden": "Peilgebied_Dellen.shp"}

vorm_mapping = dict(rond=1,
                    driehoekig=1,
                    rechthoekig=3,
                    eivormig=1,
                    ellips=1,
                    Paraboolvormig=1,
                    trapeziumvormig=1,
                    heul=1,
                    muil=3,
                    langwerpig=1,
                    scherp=1,
                    onbekend=99,
                    overig=99)

ruwheid_mapping = {"A1": 0.03,
                   "A2": 0.03,
                   "A2 Boot": 0.03,
                   "A3": 0.03,
                   "B1": 0.02,
                   "B2": 0.022,
                   "C1": 0.02,
                   "C2b": 0.022,
                   "DERDEN": 0.03,
                   "GEEN": 0.07}

for key, item in invoerbestanden.items():
    if not beheerregister.joinpath(item).exists():
        print(f"bestand voor {key} bestaat niet: {beheerregister.joinpath(item)}")
        
for key, item in modelbestanden.items():
    if not excelbestanden.joinpath(item).exists():
        print(f"bestand voor {key} bestaat niet: {excelbestanden.joinpath(item)}")
        
print("Paden succesvol ingesteld.")

D:\GITHUB\PILOT_HENA\modelbouwscript\rtc_generic_files
Paden succesvol ingesteld.


## Inlezen beheerregister in HyDAMO

### Aanmaken HyDAMO object

Alle benodigde modules worden geimporteerd en het HYDAMO object wordt aangemaakt bij het uitvoeren van onderstaand code-blok

In [46]:
from delft3dfmpy import DFlowFMModel, HyDAMO, Rectangular, DFlowFMWriter
from delft3dfmpy import DFlowRRModel, DFlowRRWriter
from delft3dfmpy.datamodels.common import ExtendedGeoDataFrame
import hydrotools
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString

hydamo = HyDAMO(extent_file=str(beheerregister.joinpath(invoerbestanden["modelgebied"])))

### Toevoegen waterlopen

De waterlopen (branches) worden toegevoegd met volgend code-blok:
* de branches worden toegekend aan het hydamo-object
* een HyDAMO ruwheidscode (4 = manning) wordt toegekend
* de eindpunten van de branches worden gesnapped binnen een tolerantie van 1m. De coordinaten worden ook afgerond op 1m.

In [47]:
# gdf = gpd.read_file(beheerregister.joinpath(invoerbestanden["branches"]))
# gdf.columns = gdf.columns.str.lower()
# print(gdf.columns)

gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["branches"]),
                           hydamo_attribute="branches",
                           index_col="ovkident",
                           keep_columns=["code",
                                         "bodembreedte",
                                         "bodemhoogte_bov",
                                         "bodemhoogte_ben",
                                         "bovenbreedte",
                                         "taludhellinglinkerzijde",
                                         "avvtalur",
                                         "taludhellingrechterzijde",
                                         "werkcode",
                                         "objectid"],
                           column_mapping={"ovkident": "code",
                                           "avvboddr": "bodembreedte",
                                           "avvhobos": "bodemhoogte_bov",
                                           "avvhobes": "bodemhoogte_ben",
                                           "iws_bovenb": "bovenbreedte",
                                           "avvtalul": "taludhellinglinkerzijde",
                                           "avvtalur": "taludhellingrechterzijde"}
                           )

gdf.loc[:, "ruwheidstypecode"] = 2
gdf.loc[:, "ruwheidswaarde"] = gdf.apply((lambda x: ruwheid_mapping[x["werkcode"]]), axis=1)

#print(beheerregister.joinpath(invoerbestanden["branches"]))
#print("kolommen in geodataframe: " + gdf.columns)

#hydamo.branches = hydrotools.snap_ends(hydamo.branches, tolerance=1, digits=1)

# verplaatsen eindpunten
#move_lines_gdf = gpd.read_file(data_path.joinpath("shp","verplaats_eind_nodes.shp"))
#gdf = hydrotools.move_end_nodes(gdf, move_lines_gdf, threshold=1)

hydamo.branches.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)

#hydamo.branches = hydrotools.snap_ends(hydamo.branches, tolerance=1, digits=1)

#hydamo.branches = hydamo.branches.loc[~hydamo.branches.index.isin(["OAF016902",
 #                                                                  "OAF016901"])]

### Aanmaken principeprofielen
Als terugvaloptie voor branches zonder profielen schrijven we principeprofielen weg die we later in deze notebook gebruiken

In [48]:
principe_profielen_bov_df = hydrotools.get_trapeziums(gdf,
                                                  "code",
                                                  "bodembreedte",
                                                  "bodemhoogte_bov",
                                                  "bovenbreedte",
                                                  "taludhellinglinkerzijde",
                                                  "taludhellingrechterzijde",
                                                  "ruwheidstypecode",
                                                  "ruwheidswaarde")

principe_profielen_ben_df = hydrotools.get_trapeziums(gdf,
                                                  "code",
                                                  "bodembreedte",
                                                  "bodemhoogte_ben",
                                                  "bovenbreedte",
                                                  "taludhellinglinkerzijde",
                                                  "taludhellingrechterzijde",
                                                  "ruwheidstypecode",
                                                  "ruwheidswaarde")

principe_profielen_bov_df.to_csv(excelbestanden.joinpath("principe_profielen_bovenstrooms.csv"))
principe_profielen_ben_df.to_csv(excelbestanden.joinpath("principe_profielen_benedenstrooms.csv"))

### Aanmaken laterale instroompunten RR

In [49]:
hydamo.laterals.read_shp(beheerregister.joinpath(data_path,'shp/1DLateralInflowPoints.shp'),
                         column_mapping={"OBJECTID":"code",
                                        "X":"X",
                                        "Y":"Y"})
hydamo.laterals.snap_to_branch(hydamo.branches, snap_method='overal', maxdist=500)
hydamo.laterals.dropna(axis=0, inplace=True, subset=['branch_offset'])
     

### Toevoegen yz-profielen
De yz-profielen worden ingeladen:
* De profielpunten worden geopend
* de punten worden geordend en geconverteerd naar polylinen met een xyz coordinaten
* toekennen ruwheid (manning = 35)
* lijnen langer dan 500m worden weggegooid, omdat er soms kademuren PRO_TYPE = PRO hebben gekregen
* lijnen worden toegekend aan het HyDAMO object
* lijnen die niet snappen met de branches, worden weggegooid

ToDo:
* verbeteren toekenning ruwheid

In [50]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["profielpunten"]),
                           "crosssections",
                           attribute_filter={"osmomsch": ["Z1"]},  #we nemen alleen profielen van het type Z1 (vaste bodem) 
                           column_mapping={"CODE": "code",
                                           "IWS_VOLGNR": "order",
                                           "OSMOMSCH": "category",
                                           "iws_hoogte": "z"},
                           z_coord=True
                           )
print(gdf["code"])
grouper = gdf.groupby("code")
profiles = dict()

for code, prof_gdf in grouper:
    if len(prof_gdf) > 1: 
        prof_gdf = prof_gdf.sort_values("order")  #Staan ook punten met zelfde order
        first_point = prof_gdf.iloc[0]["geometry"]
        cum_dist = -1
        line = []
        for idx, (_, row) in enumerate(prof_gdf.iterrows()):
            geom = row["geometry"]
            distance = geom.distance(first_point)
            if distance > cum_dist:
                cum_dist = distance
                line += [(geom.x, geom.y, row["z"])]

        profiles[code] = [code, LineString(line)]
    else:
        print(f"Waarschuwing: {code} bevat slechts één punt")
profiles_gdf = gpd.GeoDataFrame.from_dict(profiles,
                                          orient="index",
                                          columns=["code",
                                                   "geometry"]
                                          )

profiles_gdf["ruwheidstypecode"] = 4
profiles_gdf["ruwheidswaarde"] = 35
profiles_gdf["codegerelateerdobject"] = None

profiles_gdf = profiles_gdf.loc[profiles_gdf["geometry"].length < 500]

hydamo.crosssections.set_data(profiles_gdf,
                              index_col="code",
                              check_columns=True,
                              check_geotype=True)

hydamo.crosssections.snap_to_branch(hydamo.branches, snap_method="intersecting")
hydamo.crosssections.dropna(axis=0, inplace=True, subset=["branch_offset"])

0          20030408-7
5          20030404-5
7          20030312-7
8          20030310-9
9          20030211-3
            ...      
3980    DP20151028-11
3982    DP20151028-15
3985    DP20151028-12
3986    DP20151028-11
3987    DP20151028-11
Name: code, Length: 2371, dtype: object
Waarschuwing: 20060329-1372 bevat slechts één punt
Waarschuwing: 20060329-1373 bevat slechts één punt
Waarschuwing: DP20130329-36 bevat slechts één punt
Waarschuwing: DP20140224-06 bevat slechts één punt
Waarschuwing: DP20151026-21 bevat slechts één punt
Waarschuwing: LT0196180904113302 bevat slechts één punt


### Toevoegen geparameteriseerde profielen

Per brug moet er een profiel worden toegevoegd

In [51]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["bruggen"]),
                           "parametrised_profiles",
                           column_mapping={
                                           "KBRIDENT":"code",
                                           "KBRBHBO": "bodemhoogtebovenstrooms",
                                           "KBRBREED": "bodembreedte",
                                           "KBRHOBO": "hoogteinsteeklinkerzijde",
                                           })

grouper = gdf.groupby("code")
data = {"code": [code for code, frame in grouper],
        "bodemhoogtebovenstrooms": [frame["bodemhoogtebovenstrooms"].values[0]
                                    for code, frame in grouper],
        "bodembreedte": [frame["bodembreedte"].values[0]
                         for code, frame in grouper],
        "hoogteinsteeklinkerzijde": [frame["hoogteinsteeklinkerzijde"].values[0]
                                     for code, frame in grouper],
        "geometry": [frame["geometry"].values[0] for code, frame in grouper]
        }

gdf = gpd.GeoDataFrame(data)
gdf["geometry"] = gdf.apply((lambda x: LineString([[x["geometry"].x,
                                                    x["geometry"].y],
                                                  [x["geometry"].x+1,
                                                   x["geometry"].y+1]])),
                            axis=1)

gdf["codegerelateerdobject"] = gdf["code"].copy()
gdf["code"] = [f"PRO_{code}" for code in gdf["code"]]
gdf["hoogteinsteekrechterzijde"] = gdf["hoogteinsteeklinkerzijde"]
gdf["bodemhoogtebenedenstrooms"]= gdf["bodemhoogtebovenstrooms"]
gdf["ruwheidswaarde"] = 15
gdf["ruwheidstypecode"] = 4
gdf["taludhellinglinkerzijde"] = 1
gdf["taludhellingrechterzijde"] = 1

#vervang nan-waarde voor bodembreedte door  defaultwaarde 1
gdf.loc[gdf["bodembreedte"].isna(), "bodembreedte"] = 1

hydamo.parametrised_profiles.set_data(gdf,
                                      index_col="code",
                                      check_columns=True,
                                      check_geotype=False)

### Toevoegen bruggen
Per brug moet er een profiel worden toegevoegd

In [52]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["bruggen"]),
                           "bridges",
                           column_mapping={
                                           "KBRIDENT":"code",
                                           "KBRLENGT": "lengte",
                                           "KBRHOBO": "hoogtebovenzijde",
                                           "KBRBHBO": "hoogteonderzijde"})

grouper = gdf.groupby("code")
data = {"code": [code for code, frame in grouper],
        "hoogteonderzijde": [frame["hoogteonderzijde"].values[0] for code, frame in grouper],
        "hoogtebovenzijde": [frame["hoogtebovenzijde"].values[0] for code, frame in grouper], 
        "lengte": [frame["lengte"].values[0] for code, frame in grouper],
        "geometry": [frame["geometry"].values[0] for code, frame in grouper]
        }
gdf = gpd.GeoDataFrame(data)

#gdf["hoogtebovenzijde"] = gdf["hoogteonderzijde"] + 1

gdf["dwarsprofielcode"] = gdf["code"]
gdf["intreeverlies"] = 0.7
gdf["uittreeverlies"] = 0.7
gdf["ruwheidstypecode"] = 4
gdf["ruwheidswaarde"] = 70

hydamo.bridges.set_data(gdf, index_col="code", check_columns=True, check_geotype=False)
hydamo.bridges.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=1)
hydamo.bridges.dropna(axis=0, inplace=True, subset=["branch_offset"])
print("Aantal bruggen binnen snapping distance: " + str(len(hydamo.bridges)))
hydamo.bridges.dropna(axis=0, inplace=True, subset=["hoogteonderzijde"])
print("Aantal bruggen toegevoegd aan het model: " + str(len(hydamo.bridges)))


Aantal bruggen binnen snapping distance: 7
Aantal bruggen toegevoegd aan het model: 7


### Toevoegen duikers en siphons
duikers en sifons worden als cuverts aan het HyDAMO object toegekend

ToDo
* vorm moet naar HyDAMO codering worden omgeschreven

In [53]:
culverts_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["duikers"]),
                                    "culverts",
                                    column_mapping={
                                                    "KDUIDENT": "code",
                                                    "KDULENGT": "lengte",
                                                    "KDUHGA1": "hoogteopening",
                                                    "KDUBREED": "breedteopening",
                                                    "KDUBOKBE": "hoogtebinnenonderkantbenedenstrooms",
                                                    "KDUBOKBO": "hoogtebinnenonderkantbovenstrooms",
                                                    "KDUVORM": "vormcode"})

siphons_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["sifons"]),
                                   "culverts",
                                   column_mapping={
                                                   "KSYIDENT": "code",
                                                   "IWS_LENGTE": "lengte",
                                                   "KSYHGA1": "hoogteopening",
                                                   "KSYBREED": "breedteopening",
                                                   "IWS_HBOKBE":"hoogtebinnenonderkantbenedenstrooms",
                                                   "IWS_HBOKBO":"hoogtebinnenonderkantbovenstrooms",
                                                   "KSYVORM":"vormcode"})

gdf = gpd.GeoDataFrame(pd.concat([culverts_gdf,siphons_gdf], ignore_index=True))

gdf.loc[gdf['vormcode'].isnull(), 'vormcode'] = 'onbekend'
gdf.loc[:, 'vormcode'] = gdf.apply((lambda x: vorm_mapping[x['vormcode'].lower()]), axis=1)

gdf["intreeverlies"] = 0.7
gdf["uittreeverlies"] = 0.7
gdf["ruwheidswaarde"] = 70
gdf["ruwheidstypecode"] = 4

hydamo.culverts.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)
hydamo.culverts.snap_to_branch(hydamo.branches, snap_method="centroid", maxdist=1)

print("Number of culverts in datasource is " + str(len(hydamo.culverts)))
hydamo.culverts.dropna(axis=0, inplace=True, subset=["branch_offset"])
print("Number of culverts withing snapping range is " + str(len(hydamo.culverts)))
hydamo.culverts.dropna(axis=0, inplace=True, subset=["hoogtebinnenonderkantbovenstrooms"])
hydamo.culverts.dropna(axis=0, inplace=True, subset=["hoogtebinnenonderkantbenedenstrooms"])
print("Number of culverts written to model is " + str(len(hydamo.culverts)))

hydamo.culverts.loc[hydamo.culverts["hoogteopening"].isna(), "hoogteopening"] = 0.5
hydamo.culverts.loc[hydamo.culverts["breedteopening"].isna(), "breedteopening"] = 0.5
hydamo.culverts.loc[hydamo.culverts["lengte"].isna(), "lengte"] = 20


Number of culverts in datasource is 528
Number of culverts withing snapping range is 149
Number of culverts written to model is 147


### Toevoegen stuwen

In [54]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["stuwen"]),
                           "weirs",
                           column_mapping={"kstident": "code",
                                           "kstsoort": "soortstuwcode",
                                           "kstkrubr": "laagstedoorstroombreedte",
                                           "kstmikho": "laagstedoorstroomhoogte",
                                           "kstmaxkr": "hoogstedoorstroomhoogte",
                                           "kstregel": "soortregelbaarheidcode",
                                           "HEA_GPGIDE":"gpg_id"},
                            keep_columns=["gpg_id"])

gdf["afvoercoefficient"] = 1

hydamo.weirs.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)
hydamo.weirs.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=1)
hydamo.weirs.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Toevoegen gemalen

In [55]:
pumps_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["gemalen"]),
                                 "pumps",
                                 column_mapping={
                                     "KGMIDENT": "code",
                                     "KGMMACAP": "maximalecapaciteit"})

gemalen_gdf = pumps_gdf.copy()
pumps_gdf["codegerelateerdobject"] = pumps_gdf["code"].copy()
pumps_gdf["code"] = [f"PMP_{code}" for code in pumps_gdf["code"]]
hydamo.gemalen.set_data(gemalen_gdf,
                        index_col="code",
                        check_columns=True,
                        check_geotype=True)
hydamo.gemalen.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=1)
hydamo.gemalen.dropna(axis=0, inplace=True, subset=["branch_offset"])
hydamo.pumps.set_data(pumps_gdf,
                      index_col="code",
                      check_columns=True,
                      check_geotype=True)
hydamo.pumps.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=1)
hydamo.pumps.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Toevoegen afsluitmiddelen

In [56]:
gdf = pd.DataFrame({"code": [],
                    "soortafsluitmiddelcode": [],
                    "codegerelateerdobject": []})

hydamo.afsluitmiddel.set_data(gdf, index_col="code")

### Toevoegen sturing

In [57]:
gpg_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["peilgebieden"]),
                                 "sturing",
                                 column_mapping={"GPGIDENT": "code",
                                                 "GPGZMRPL": "zomerpeil",
                                                 "GPGWNTPL": "winterpeil"
                                                 },
                                  keep_columns=["code",
                                         "zomerpeil",
                                         "winterpeil",
                                              ])

#print(gpg_gdf)

gpg_gdf["codegerelateerdobject"] = [code.replace("GPG", "KGM") for code in gpg_gdf["code"]]
gpg_gdf = gpg_gdf[gpg_gdf["codegerelateerdobject"].isin(hydamo.gemalen["code"])]

con_gdf = hydamo.gemalen[hydamo.gemalen["code"].isin(gpg_gdf["codegerelateerdobject"])]
con_gdf["objectid"] = [code for code in con_gdf["code"]]

gdf = con_gdf.merge(gpg_gdf, how='left', left_on='objectid', right_on='codegerelateerdobject', suffixes=('', '_merge'))

gdf["streefwaarde"] = gdf["zomerpeil"]
gdf["bovenmarge"] = gdf["zomerpeil"] + 0.05
gdf["ondermarge"] = gdf["zomerpeil"] - 0.05
gdf["doelvariabelecode"] = 1

hydamo.sturing.set_data(gdf, index_col="code")

print("sturing is ingesteld op basis van de onderstaande tabel:")

gdf


sturing is ingesteld op basis van de onderstaande tabel:


,code,maximalecapaciteit,geometry,branch_id,branch_offset,objectid,code_merge,winterpeil,zomerpeil,geometry_merge,codegerelateerdobject,streefwaarde,bovenmarge,ondermarge,doelvariabelecode
0,KGM-O-11860,13.0,POINT (258504.964 585292.983),OAF-O-00782,356.855949,KGM-O-11860,GPG-O-11860,-3.9,-3.45,"POLYGON ((258291.600 586725.971, 258314.144 58...",KGM-O-11860,-3.45,-3.40,-3.50,1
1,KGM-O-11730,320.0,POINT (258080.954 584320.392),OAF-O-02479,11.899886,KGM-O-11730,GPG-O-11730,-3.6,-3.30,"MULTIPOLYGON (((256923.180 585095.757, 256911....",KGM-O-11730,-3.30,-3.25,-3.35,1


### Filteren

We maken een subset van branches op basis van waarden uit een gegeven veld. In dit geval worden alle takken waarvoor geldt OBJECTID = 1 opgenomen in de modelschematisatie. Idem voor alle kunstwerken + profielen die naar deze branches zijn gesnapped. Om het resultaat te beoordelen exporteren we alles naar shape-files. We slaan het hydamo object ook op als "pickle", zodat we bovenstaande stappen niet elke keer hoeven te herhalen

In [58]:
hydamo = hydrotools.filter_model(hydamo, attribute_filter={"OBJECTID": 1})
#hydrotools.export_shapes(hydamo, path=Path(r"./hydamo_shp/dellen"))
hydrotools.save_model(hydamo, file_name=Path(r"./hydamo_model/dellen.pickle"))


## Converteren naar DFM

### Aanmaken dfm-klassen

In [59]:
dfmmodel = DFlowFMModel()
drrmodel = DFlowRRModel()

### Inlezen kunstwerken en laterale instromingen

In [60]:
dfmmodel.structures.io.weirs_from_hydamo(hydamo.weirs,
                                         yz_profiles=hydamo.crosssections,
                                         parametrised_profiles=hydamo.parametrised_profiles)

dfmmodel.structures.io.culverts_from_hydamo(hydamo.culverts,
                                            hydamo.afsluitmiddel)

dfmmodel.structures.io.bridges_from_hydamo(hydamo.bridges,
                                           yz_profiles=hydamo.crosssections,
                                           parametrised_profiles=hydamo.parametrised_profiles)

#for lateral in hydamo.laterals.itertuples():
#    dfmmodel.external_forcings.laterals[lateral.code] = {
#        'branchid': lateral.branch_id,
#        'branch_offset':str(lateral.branch_offset)
#    }
#    drrmodel.external_forcings.add_boundary_node(lateral.code, lateral.X, lateral.Y)


dfmmodel.structures.io.orifices_from_hydamo(hydamo.orifices)

dfmmodel.structures.io.pumps_from_hydamo(pompen=hydamo.pumps,
                                         sturing=hydamo.sturing,
                                         gemalen=hydamo.gemalen)

2021-08-26 22:18:19,711 - delft3dfmpy.converters.hydamo_to_dflowfm - hydamo_to_dflowfm - INFO - Currently only simple weirs can be applied. From Hydamo the attributes 'laagstedoorstroomhoogte' and 'kruinbreedte' are used to define the weir dimensions.


INFO:delft3dfmpy.converters.hydamo_to_dflowfm:Currently only simple weirs can be applied. From Hydamo the attributes 'laagstedoorstroomhoogte' and 'kruinbreedte' are used to define the weir dimensions.


### Toevoegen sturing


#### Dit blok is komen te vervallen met de introductie van RTC. Sowieso draaien tijdsafhankelijke kruinhoogtes nog niet in de GUI

kst_gestuurd = hydamo.weirs.loc[
    hydamo.weirs['soortregelbaarheidcode'] != 'vast'
    ]['gpg_id']

gpg_gdf = gpd.read_file(beheerregister.joinpath(invoerbestanden["peilgebieden"]))
gpg_gdf.set_index("GPGIDENT", inplace=True)

ts_start_datetime = pd.Timestamp(year=2010, month=1, day=1)
ts_end_datetime = pd.Timestamp(year=2030, month=1, day=1)
for kst_id, gpg_id in kst_gestuurd.iteritems():
    if gpg_id in gpg_gdf.index:
        zomerpeil = gpg_gdf.loc[gpg_id]["GPGZMRPL"]
        winterpeil = gpg_gdf.loc[gpg_id]["GPGWNTPL"]
        series = hydrotools.generate_target_series(target_summer=zomerpeil,
                                                       target_winter=winterpeil,
                                                       start_datetime=ts_start_datetime,
                                                       end_datetime=ts_end_datetime)
        dfmmodel.external_forcings.set_structure_series(kst_id,
                                                        'weir',
                                                        'crestLevel',
                                                        series)

### Aanmaken 1d netwerk

In [61]:
dfmmodel.network.set_branches(hydamo.branches)
dfmmodel.network.generate_1dnetwork(one_d_mesh_distance=100.0, seperate_structures=True)

2021-08-26 22:18:19,881 - delft3dfmpy.core.dfm - dfm - WARNING - Some structures are not linked to a branch.


2021-08-26 22:18:19,885 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF-O-00002 at: [0.0, 44.941500000000005, 71.82240223650987], due to the structures at [-0.001, 22.061, 67.822, 71.82340223650988].


INFO:delft3dfmpy.core.dfm:Added 1d mesh nodes on branch OAF-O-00002 at: [0.0, 44.941500000000005, 71.82240223650987], due to the structures at [-0.001, 22.061, 67.822, 71.82340223650988].


2021-08-26 22:18:19,888 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF-O-00059 at: [0.0, 985.2394999999999, 1046.7060000000001, 1083.9012346060038], due to the structures at [-0.001, 202.42, 317.606, 532.454, 949.228, 1021.251, 1072.161, 1083.9022346060037].


INFO:delft3dfmpy.core.dfm:Added 1d mesh nodes on branch OAF-O-00059 at: [0.0, 985.2394999999999, 1046.7060000000001, 1083.9012346060038], due to the structures at [-0.001, 202.42, 317.606, 532.454, 949.228, 1021.251, 1072.161, 1083.9022346060037].


...

INFO:delft3dfmpy.core.dfm:Added 1d mesh nodes on branch OAF-O-00957 at: [0.0, 221.44400000000002, 247.5970873977168], due to the structures at [-0.001, 206.193, 236.695, 247.5980873977168].


2021-08-26 22:18:19,934 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF-O-01042 at: [0.0, 23.267, 91.84008287618335, 142.22916575236667], due to the structures at [-0.001, 4.983, 41.551, 142.12916575236667, 142.23016575236667].


INFO:delft3dfmpy.core.dfm:Added 1d mesh nodes on branch OAF-O-01042 at: [0.0, 23.267, 91.84008287618335, 142.22916575236667], due to the structures at [-0.001, 4.983, 41.551, 142.12916575236667, 142.23016575236667].


2021-08-26 22:18:19,937 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF-O-02383 at: [0.0, 56.6605, 95.52468010517553], due to the structures at [-0.001, 45.06, 68.261, 95.52568010517554].


INFO:delft3dfmpy.core.dfm:Added 1d mesh nodes on branch OAF-O-02383 at: [0.0, 56.6605, 95.52468010517553], due to the structures at [-0.001, 45.06, 68.261, 95.52568010517554].


### Toevoegen cross-sections

In [62]:
dfmmodel.crosssections.io.from_hydamo(
    dwarsprofielen=hydamo.crosssections,
    parametrised=hydamo.parametrised_profiles,
    branches=hydamo.branches
)

print(f"{len(dfmmodel.crosssections.get_branches_without_crosssection())} branches are still missing a cross section.")
print(f"{len(dfmmodel.crosssections.get_structures_without_crosssection())} structures are still missing a cross section.")

2021-08-26 22:18:20,050 - delft3dfmpy.io.dfmreader - dfmreader - INFO - Before adding the number of branches without cross section is: 172.


INFO:delft3dfmpy.io.dfmreader:Before adding the number of branches without cross section is: 172.


2021-08-26 22:18:20,256 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'dwarsprofielen' the number of branches without cross section is: 52.


INFO:delft3dfmpy.io.dfmreader:After adding 'dwarsprofielen' the number of branches without cross section is: 52.


No parametrised crossections available for branches.
2021-08-26 22:18:20,258 - delft3dfmpy.io.dfmreader - dfmreader - INFO - Before adding the number of structures without cross section is: 7.


...

2021-08-26 22:18:20,269 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'dwarsprofielen' the number of branches without cross section is: 7.


INFO:delft3dfmpy.io.dfmreader:After adding 'dwarsprofielen' the number of branches without cross section is: 7.


2021-08-26 22:18:20,277 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'normgeparametriseerd' the number of structures without cross section is: 0.


INFO:delft3dfmpy.io.dfmreader:After adding 'normgeparametriseerd' the number of structures without cross section is: 0.


52 branches are still missing a cross section.
0 structures are still missing a cross section.


### Toevoegen principeprofielen op takken die nog niet over een profiel beschikken

In [63]:
if len(dfmmodel.crosssections.get_branches_without_crosssection()) > 0:        
    print("adding trapezium profiles on branches with missing crosssections.")
    #siebe 22-6-2021: onderscheid boven- en benedenstrooms profiel. Vanaf nu twee dataframes meegeven
    # Bram 4-7-2021 maximum Flowwidth wordt 300 in output met closed=False!? -> closed=True
    dfmmodel = hydrotools.add_trapeziums(dfmmodel, principe_profielen_bov_df, principe_profielen_ben_df, closed=True)

print(f"{len(dfmmodel.crosssections.get_branches_without_crosssection())} number of branches remain with no cross section due to missing data (no bedlevels).")

adding trapezium profiles on branches with missing crosssections.
0 number of branches remain with no cross section due to missing data (no bedlevels).


### Toevoegen vierkante profielen op takken met duiker en lengte vergelijkbaar met duikerlengte

In [64]:
# Bram 4-7-2021: Ronde profielen werkt niet -> Ook in GUI lukt het niet 
# om de dwarsdoorsnede door validatie heen te krijgen.
# Verder lijkt D-FM moeite te hebben met gedeelte dwarsprofielen ('shared definitions')
# Deze routine maakt daarom unieke definities aan per dwarsdoorsnede locatie

crs_def_id = 0 # start uniek ID dwarsdoorsnede definitie

# loop door branches_id's met missende crs
for branch_id in dfmmodel.crosssections.get_branches_without_crosssection(): 
    
    # als de branch in de culvert branch lijst staat (branch heeft culvert)
    if branch_id in hydamo.culverts.branch_id.to_list(): 

        culvert_idx = hydamo.culverts.branch_id.to_list().index(branch_id) # index location van de duiker (niet nodig?)
        culvert = hydamo.culverts.iloc[[culvert_idx]]
        
        # lengtes van duiker en watergang
        culvert_length = culvert['lengte'][0]
        branch_length = dfmmodel.network.branches.geometry[branch_id].length
        
        if culvert_length > 0.5 * branch_length: # check of de duiker meer dan 50% van de watergang beslaat
        
            # get culvert attributes
            culvert_diameter = culvert['hoogteopening'][0]
            roughnesstype = int(culvert['ruwheidstypecode'][0])
            roughness = culvert['ruwheidswaarde'][0]
            bob_boven = round(culvert['hoogtebinnenonderkantbovenstrooms'][0],3)
            bob_beneden = round(culvert['hoogtebinnenonderkantbenedenstrooms'][0],3)

            # maak unieke namen voor dwarsdoorsnede profiel definities
            crs_def_boven_name = f'rect_{crs_def_id}'
            crs_def_beneden_name = f'rect_{crs_def_id+1}'
            crs_def_id += 2

            # Voeg vierkante profiel definities toe
            dfmmodel.crosssections.add_rectangle_definition(name= crs_def_boven_name, 
                                                         height = culvert_diameter, 
                                                         width = culvert_diameter, 
                                                         closed = False,
                                                         roughnesstype = roughnesstype, 
                                                         roughnessvalue = roughness)
            
            dfmmodel.crosssections.add_rectangle_definition(name= crs_def_beneden_name, 
                                                         height = culvert_diameter, 
                                                         width = culvert_diameter, 
                                                         closed = False,
                                                         roughnesstype = roughnesstype, 
                                                         roughnessvalue = roughness)


            # Voeg dwarsdoorsnede locaties toe
            dfmmodel.crosssections.add_crosssection_location(branchid = branch_id, 
                                                             chainage = 0.1, 
                                                             definition = crs_def_boven_name, 
                                                             shift = bob_boven)
            
            dfmmodel.crosssections.add_crosssection_location(branchid = branch_id,
                                                             chainage = round((branch_length-0.1),4), 
                                                             definition = crs_def_beneden_name, 
                                                             shift = bob_beneden)

            print(f'added crs for branch {branch_id} based on culvert {culvert["code"][0]}')

print(f"{len(dfmmodel.crosssections.get_branches_without_crosssection())} number of branches remain with no cross section due to missing data.")
print('Still missing:', '\n'.join(dfmmodel.crosssections.get_branches_without_crosssection()))

## if branch contains culvert
## if abs(length(branch) - (length(culvert))) <= 100 then
##    we zitten in een kunstwerkvak
##    maak rond profiel conform duikerprofile
##    push rond profile naar dfmodel middels een dataframe


0 number of branches remain with no cross section due to missing data.
Still missing: 


In [65]:
### alle overgebleven lege takken verwijderen

### Randvoorwaarden toevoegen

In [66]:
rvw_df = pd.read_excel(excelbestanden.joinpath(modelbestanden["randvoorwaarden"]))

for _,row in rvw_df.iterrows():
    branch_id = row["BRANCH_ID"]
    pt = hydamo.branches.loc[branch_id]["geometry"].coords[int(row["COORD"])]
    series = pd.Series(data=[-3.0, -3.0],
                       index=[pd.Timestamp(year=2000, month=1, day=1),
                              pd.Timestamp(year=2100, month=1, day=1)]
                       )
    dfmmodel.external_forcings.add_boundary_condition(name=f"BND_{branch_id}",
                                                      pt=pt,
                                                      bctype=row["TYPE"],
                                                      series=series)
    print(series)

print("Boundary conditions written:")
rvw_df


2000-01-01   -3.0
2100-01-01   -3.0
dtype: float64
Boundary conditions written:


,BRANCH_ID,COORD,TYPE,Unnamed: 3
0,OAF-O-02479,-1,waterlevel,-3


In [67]:
rvw_df = pd.read_excel(excelbestanden.joinpath(modelbestanden["randvoorwaarden"]))

for _,row in rvw_df.iterrows():
    branch_id = row["BRANCH_ID"]
    pt = hydamo.branches.loc[branch_id]["geometry"].coords[int(row["COORD"])]
    series = pd.Series(data=[-4.0, -4.0],
                       index=[pd.Timestamp("2000-01-01"),
                              pd.Timestamp("2100-01-01")]
                       )

### Wegschrijven model

In [68]:
dimr_path = r"c:\Program Files\Deltares\D-HYDRO Suite 1D2D (Beta) (0.9.11.53157)\plugins\DeltaShell.Dimr\kernels\x64\dimr\scripts\run_dimr.bat"
start_datetime = pd.Timestamp('2000-01-01 00:00:00')
end_datetime = start_datetime + pd.Timedelta(days=6)
mdu_file = "dellen"

dfmmodel.mdu_parameters["refdate"] = int(start_datetime.strftime("%Y%m%d"))
dfmmodel.mdu_parameters["tstart"] = 0.0 * 3600
dfmmodel.mdu_parameters["tstop"] = 10 * 24 * 3600
dfmmodel.mdu_parameters["hisinterval"] = "600. 0. 0."
dfmmodel.mdu_parameters["mapinterval"] = "600. 0. 0."
dfmmodel.mdu_parameters["wrirst_bnd"] = 0
dfmmodel.mdu_parameters["cflmax"] = 0.7
dfmmodel.mdu_parameters["outputdir"] = "output"
dfmmodel.dimr_path = dimr_path

# adding observation point file for rtc that is created a below
dfmmodel.mdu_parameters['ObsFile'] = "observationPoints.ini" 

#fm_writer = DFlowFMWriter(dfmmodel, output_dir=r"dellen", name="dellen")
fm_writer = DFlowFMWriter(dfmmodel, output_dir=outputdir, name="dellen")

fm_writer.objects_to_ldb()
fm_writer.write_all()

#2021-08-26 Siebe Bosch: we need to remove the dimr_config.xml since that file will be written one directory up
if os.path.isfile(os.path.join(fmdir,"dimr_config.xml")) == True:
    os.remove(os.path.join(fmdir,"dimr_config.xml"))
    print("removed dimr_config.xml from the fm dir since it will be defined on a higher level")

drrmodel.d3b_parameters['Timestepsize'] = 300
drrmodel.d3b_parameters['StartTime'] = start_datetime.strftime("%Y/%m/%d;%H:%M:%S") # should be equal to refdate for D-HYDRO
drrmodel.d3b_parameters['EndTime'] = end_datetime.strftime("%Y/%m/%d;%H:%M:%S")
drrmodel.d3b_parameters['RestartIn'] = 0
drrmodel.d3b_parameters['RestartOut'] = 0
drrmodel.d3b_parameters['RestartFileNamePrefix'] ='Test'
drrmodel.d3b_parameters['UnsaturatedZone'] = 1
drrmodel.d3b_parameters['UnpavedPercolationLikeSobek213']=-1
drrmodel.d3b_parameters['VolumeCheckFactorToCF']=100000
drrmodel.dimr_path = dimr_path

rr_writer = DFlowRRWriter(drrmodel,
                          output_dir=r"dellen",
                          name="dellen")
rr_writer.write_coupling()


removed dimr_config.xml from the fm dir since it will be defined on a higher level


# RTC
#### this code was derived from the script as created in TKI2 by RHDVH

In [69]:
# RTC on/off
RTC = 1

#### General RTC settings

In [70]:
#standard rtc settings
rtc_update_int = int(dfmmodel.mdu_parameters['hisinterval'].split('.')[0]) # update interval rtc module with fm module in seconds (controller frequency)

#Rules on/off
pidrule = 0
intervalrule = 0
timeseriesrule = 1 #wat betekent dit?

#standard settings PID
settingMaxspeed = 0.01     # m/s per timestep
kp = 1                     # K proportional
ki = 0.01                  # K integral
kd = 0.0                   # K differential

#standard interval rule settings
deadbandSetpointAbsolute = 0.1    # bandwith around setpoint where no action is taken.
settingMaxspeed_int = 0.001       # m/s per timestep
settingMaxspeed_int_pumps = 1     # m3/s per timestep
#settingMaxStep = 0.001           # for constant setpoint (not yet avaiable)



#### List of PID, interval and time rule controlled structures 

In [71]:
#extra PID weirs (beside all described in HyDAMO)
weirs_auto = []
#weirs_auto = [\"KST-222\",\"KST-358\",\"KST-246\",\"KST-210\"]
              
#interval weirs
interval_weirs = []
#interval_weirs = [\"KST-222\",\"KST-358\",\"KST-246\",\"KST-210\"]

#interval_pumps
interval_pumps = []
#interval_pumps = [\"KGM-3_P1\",\"KGM-3_P2\"]

#timeseries weirs
timeserie_weirs = ["KST-O-17700","KST-O-17800","KST-O-17680","KST-O-17690","KST-O-17750","KST-O-17760","KST-O-17710","KST-O-17740","KST-O-17660","KST-O-17670","KST-O-17770","KST-O-17780","KST-O-17820","KST-O-17790","KST-O-17810"]
timeserie_pumps = []

print("List of PID, interval and time rule controlled structures has been set.")


List of PID, interval and time rule controlled structures has been set.


## RTC input generation from user input

In [72]:
#write "rtc ready" folder structure and copy standard RTC files to the appropriate folder

#print("writing to folder " + outputdir)
#basedir = os.path.join("02_mdu")
#print(basedir)
if os.path.isdir(os.path.join(outputdir,"fm"))==True and RTC == 1:
    print("output dir for the fm-model is fm and will be renamed to dflowfm")
    #renaming dirs
    
    #if our fm dir already exists, remove it
    if os.path.isdir(os.path.join(outputdir,"dflowfm"))==True:
        print("directory dflowfm already exists and will be removed.")
        shutil.rmtree(os.path.join(outputdir, "dflowfm")) #removes the dir incl all files and subdirs

    #if our rtc dir already exists, remove it
    if os.path.isdir(os.path.join(outputdir,"rtc"))==True:
        print("directory rtc already exists and will be removed.")
        shutil.rmtree(os.path.join(outputdir, "rtc")) #removes the dir incl all files and subdirs
    
    
    os.rename(os.path.join(outputdir,"fm"),os.path.join(outputdir,"dflowfm"))   #looks like we're renaming the 'fm' dir to 'dflowfm' here
    os.makedirs(os.path.join(outputdir,"rtc"))  #looks like we're creating an rtc dir here

#copying generic rtc files from rtc folder
src_files = os.listdir(rtc_generic_files)

# copy each file in the source dir to the destination dir
#20210826 Siebe Bosch: one exception: the dimr_config should be copied to the output dir
for file_name in src_files:
    myresult = os.path.join(rtc_generic_files, file_name)
    if os.path.isfile(myresult):
        #print("copying file " + myresult + " to " + myresult)
        shutil.copy(myresult, rtc_results_dir)
                                  

output dir for the fm-model is fm and will be renamed to dflowfm
directory rtc already exists and will be removed.


In [73]:
# create a list of all extra automated weir codes and corresponding observation points/control groups
observationpoints = []    # voor alle automatische stuwen een meetpunt 
controlgroups = []        # voor alle automatische stuwen een controle groep
controllevel= []          # voor alle automatische stuwen een controle waterniveau

print("automatic weirs are " + str(weirs_auto))
for x in range(len(weirs_auto)):
    observationpoints.append("ObservationPoint_"+ weirs_auto[x])
    controlgroups.append("Control Group "+ weirs_auto[x])
    controllevel.append(round(hydamo.weirs.loc[weirs_auto[x]].laagstedoorstroomhoogte+0.05,2))


automatic weirs are []


#### Defining the automated weirs, required observation points and control groups list

- Observation points are created 20 m upstream of the weir (be careful that 20 meters might mean that they're on another branch. If this is the case, a correction can be made using the code below)
- Automated weirs are filtered from the data (soortregelbaarheidcode == 3 )
- Controlgroups are named after the weir id
- Control level is defined as the \"laagstedoorstroomhoogte\" of the weir + 0.05 m
- The settingMin is the weir minimum crest level. this equals to the \"laagstedoorstroomhoogte\"
- The settingMax is the weirs maximum crest level. this equals to the \"hoogstedoorstroomhoogte\"

The last two lists (settingMin and settingMax) were already created in the weir section at the top of the modelgenerator, before setting other weir crest levels to the \"streefpeil\"


In [74]:
# create list of adjustable weirs and nonadjustable weirs
weirs_regelbaar = []      # handmatig, regelbaar niet automatisch, overig
weirs_nietregelbaar = []  # vaste stuwen, stuwen met waardes NULL

#loop through weirs database and filter on automated weirs (soortregelbaarheidcode == 3 ) and other types of weirs
for i in range(len(hydamo.weirs)):
    
    print("soort regelbaarheidcode: " + hydamo.weirs.soortregelbaarheidcode[i])
    
        # fill the list of all automated weir codes from HyDAMO, corresponding observation points, control groups and control levels
    if hydamo.weirs.soortregelbaarheidcode[i] == "regelbaar, automatisch":
        print("stuw " + hydamo.weirs.code[i] + " wordt automatisch gestuurd.")
        weirs_auto.append(hydamo.weirs.code[i])
        observationpoints.append("ObservationPoint_"+ str(hydamo.weirs.code[i]))
        controlgroups.append("Control Group "+ str(hydamo.weirs.code[i]))
        controllevel.append(round(hydamo.weirs.laagstedoorstroomhoogte[i]+0.05,2))
        
        #vaste stuwen. 
    elif hydamo.weirs.soortregelbaarheidcode[i] == "vast":
        print("stuw " + hydamo.weirs.code[i] + " heeft een vaste kruin.")
        weirs_nietregelbaar.append(hydamo.weirs.code[i])
        
        #handmatige stuwen
    elif hydamo.weirs.soortregelbaarheidcode[i] == "regelbaar, niet automatisch":
        print("stuw " + hydamo.weirs.code[i] + " wordt handmatig gestuurd.")
        weirs_regelbaar.append(hydamo.weirs.code[i])
                

soort regelbaarheidcode: regelbaar, niet automatisch
stuw KST-O-17700 wordt handmatig gestuurd.
soort regelbaarheidcode: regelbaar, niet automatisch
stuw KST-O-17800 wordt handmatig gestuurd.
soort regelbaarheidcode: regelbaar, automatisch
stuw KST-O-17680 wordt automatisch gestuurd.
soort regelbaarheidcode: vast
stuw KST-O-17685 heeft een vaste kruin.
soort regelbaarheidcode: regelbaar, niet automatisch
stuw KST-O-17690 wordt handmatig gestuurd.
soort regelbaarheidcode: regelbaar, automatisch
stuw KST-O-17750 wordt automatisch gestuurd.
soort regelbaarheidcode: vast
stuw KST-O-17755 heeft een vaste kruin.
soort regelbaarheidcode: regelbaar, niet automatisch
stuw KST-O-17760 wordt handmatig gestuurd.
soort regelbaarheidcode: regelbaar, niet automatisch
stuw KST-O-17710 wordt handmatig gestuurd.
soort regelbaarheidcode: vast
stuw KST-O-17730 heeft een vaste kruin.
soort regelbaarheidcode: regelbaar, niet automatisch
stuw KST-O-17740 wordt handmatig gestuurd.
soort regelbaarheidcode: reg

Create the observation points file (ini) so all weirs can be steered using this observation point, partially this is done automatic. However, for some observation points, the chainage and branch id is manually adjused

In [75]:
# create ini file for observation points
inifile = os.path.join(outputdir, "dflowfm/observationPoints.ini")
print("inifile is " + inifile)
n = open(inifile, "w+")
#n = open("02_mdu/observationPoints.ini", "w+")

#write general header for init file
n.write("[General]"+"\n")
n.write("    fileVersion           = 2.00"+"\n")
n.write("    fileType              = obsPoint"+"\n")  
        
for i in range(len(hydamo.weirs)):# for all weirs
    if hydamo.weirs.soortregelbaarheidcode[i] == "regelbaar, automatisch":
        #write observation point per automated weir (20 m before weir on branch) to the ini file
        n.write("\n"+"[ObservationPoint]"+"\n")
        n.write("    name      = ObservationPoint_"+str(hydamo.weirs.code[i])+"\n")
        
        #if weir is on edge of branch the offset is negative. with following code this is manually corrected per weir
        if hydamo.weirs.code[i] == "KST-O-17680":
            n.write("    branchId  = OAF-O-00078"+"\n")
            n.write("    chainage  = 1"+ "\n")
                
        #for all other weirs, the observation point should be 20 meters before the weir
        else:    
            n.write("    branchId  = """+str(hydamo.weirs.branch_id[i])+"\n")
            n.write("    chainage  = """+str(round(hydamo.weirs.branch_offset[i]-20,3)) +"\n")   
    else: 
        #write observation point for other weirs (0 m before weir on branch) to the ini file in case they are needed
        n.write("\n"+"[ObservationPoint]"+"\n")
        n.write("    name      = ObservationPoint_"+str(hydamo.weirs.code[i])+"\n")
        
        #if weir is on edge of branch the offset is negative. with following code this is manually corrected per weir
        if hydamo.weirs.code[i] == "KST-358" :
            n.write("    branchId  = WL_1686"+"\n")
            n.write("    chainage  = 530"+ "\n")
                
        elif hydamo.weirs.code[i] == "KST-246" :
            n.write("    branchId  = WL_1684"+"\n")
            n.write("    chainage  = 670"+ "\n")    
        
        #correction weir KST-222 and KST-210
        elif hydamo.weirs.code[i] == "KST-222":
            n.write("    branchId  = """+str(hydamo.weirs.loc["KST-222"].branch_id)+"\n")
            n.write("    chainage  = """+str(round(hydamo.weirs.loc["KST-222"].branch_offset-20,3)) +"\n") 
        
        elif hydamo.weirs.code[i] == "KST-210":
            n.write("    branchId  = """+str(hydamo.weirs.loc["KST-210"].branch_id)+"\n")
            n.write("    chainage  = """+str(round(hydamo.weirs.loc["KST-210"].branch_offset-20,3)) +"\n") 
        
        else:
            n.write("    branchId  = """+str(hydamo.weirs.branch_id[i])+"\n")
            n.write("    chainage  = """+str(round(hydamo.weirs.branch_offset[i],3)) +"\n")

for i in range(len(hydamo.pumps)):# for pumps 
    if hydamo.pumps.code[i] in interval_pumps: 
        #write observation point for other weirs (0 m before weir on branch) to the ini file in case they are needed
        n.write("\n"+"[ObservationPoint]"+"\n")
        n.write("    name      = ObservationPoint_"+str(hydamo.pumps.code[i])+"\n")
        n.write("    branchId  = """+str(hydamo.pumps.branch_id[i])+"\n")
        n.write("    chainage  = """+str(round(hydamo.pumps.branch_offset[i]-40,3)) +"\n")
    
n.close()

inifile is D:\GITHUB\PILOT_HENA\modelbouwscript\dellen\dflowfm/observationPoints.ini


In [76]:
# creating a minimal and maximum weir crest level for all automated weir before setting the laagstedoorstroomhoogte to "streefpeil" 
settingMin = []            
settingMax = []   
settingBelow = []
settingAbove = []

#these are version specific pid weirs (only select if extra pid controlled weirs are selected)
# for i in range(len(hydamo.weirs)):
#     if hydamo.weirs.code[i] in ["KST-222","KST-358","KST-246","KST-210"]:
#         settingMin.append(round(hydamo.weirs.laagstedoorstroomhoogte[i],2))
#         settingMax.append(round(hydamo.weirs.hoogstedoorstroomhoogte[i],2)) 
    
# these are all automated weirs from Hydamo database
for i in range(len(hydamo.weirs)):
    if hydamo.weirs.soortregelbaarheidcode[i] == "regelbaar, automatisch":
        settingMin.append(round(hydamo.weirs.laagstedoorstroomhoogte[i],2))
        #settingMax.append(round(hydamo.weirs.hoogstedoorstroomhoogte[i],2)) siebe: nog debuggen

# setting limist to interval structures        
for i in range(len(hydamo.weirs)):
    if hydamo.weirs.code[i] in ["KST-222","KST-358","KST-246","KST-210"]:
        settingAbove.append(round(hydamo.weirs.laagstedoorstroomhoogte[i],2))
        #settingBelow.append(round(hydamo.weirs.hoogstedoorstroomhoogte[i],2)) siebe: nog debuggen

Creating observation point names and controlgroups for the interval and timeserie rule controlled structures. The settingsabove and below are already defined at the beginning of this script (section weirs or pumps), because the values are manually adjusted after, the original values are saved first 

In [77]:
#timeserie inputs weirs
controlgroups_time = []

# interval inputs weirs
controlgroups_int= []
observationpoints_int = []

#timeserie inputs pumps
controlgroups_time_pumps = []

print("we have " + str(len(timeserie_weirs)) + " timeseries controlled weirs")
print("we have timeseriesrule " + str(timeseriesrule))

if intervalrule == 1:    

    for x in range(len(interval_weirs)):
        controlgroups_int.append("Control Group "+str(interval_weirs[x]))
        observationpoints_int.append("ObservationPoint_"+str(interval_weirs[x]))
        
    #interval inputs pumps
    controlgroups_int_pumps= []
    observationpoints_int_pumps = []

    for x in range(len(interval_pumps)):
        controlgroups_int_pumps.append("Control Group "+str(interval_pumps[x]))
        observationpoints_int_pumps.append("ObservationPoint_"+str(interval_pumps[x]))    
    
if timeseriesrule == 1:
    
    
    for x in range(len(timeserie_weirs)):  
        print("now writing control group for timeseries controlled weir " + timeserie_weirs[x])
        controlgroups_time.append("Control Group "+ str(timeserie_weirs[x]))
    
    
    for x in range(len(timeserie_pumps)):                                 
        controlgroups_time_pumps.append("Control Group "+ str(timeserie_pumps[x]))    

we have 15 timeseries controlled weirs
we have timeseriesrule 1
now writing control group for timeseries controlled weir KST-O-17700
now writing control group for timeseries controlled weir KST-O-17800
now writing control group for timeseries controlled weir KST-O-17680
now writing control group for timeseries controlled weir KST-O-17690
now writing control group for timeseries controlled weir KST-O-17750
now writing control group for timeseries controlled weir KST-O-17760
now writing control group for timeseries controlled weir KST-O-17710
now writing control group for timeseries controlled weir KST-O-17740
now writing control group for timeseries controlled weir KST-O-17660
now writing control group for timeseries controlled weir KST-O-17670
now writing control group for timeseries controlled weir KST-O-17770
now writing control group for timeseries controlled weir KST-O-17780
now writing control group for timeseries controlled weir KST-O-17820
now writing control group for timeserie

In [78]:
#evaluate the input variables for the interval and timeserie controlled structures
print(settingAbove)
print(settingBelow)
print(observationpoints_int)
print(controlgroups_int)  
print(controlgroups_time)
print(controlgroups_time_pumps)

[]
[]
[]
[]
['Control Group KST-O-17700', 'Control Group KST-O-17800', 'Control Group KST-O-17680', 'Control Group KST-O-17690', 'Control Group KST-O-17750', 'Control Group KST-O-17760', 'Control Group KST-O-17710', 'Control Group KST-O-17740', 'Control Group KST-O-17660', 'Control Group KST-O-17670', 'Control Group KST-O-17770', 'Control Group KST-O-17780', 'Control Group KST-O-17820', 'Control Group KST-O-17790', 'Control Group KST-O-17810']
[]


#### RTC File generation

A total of five xml files need to be created to generate the RTC module and couple it to the hydraulic model. 

##### RTC module:
- rtcRuntimeConfig.xml
- rtcToolsConfig.xml
- rtcDataConfig.xml
- timeseries_import.xml 

##### Coupling file: 
- dimr_config.xml 


#### Writing rtcRuntimeConfig.xml file 

In [79]:
if RTC == 1:
    #namespaces for all other xml files
    generalname = "http://www.wldelft.nl/fews"
    xsi_name = "http://www.w3.org/2001/XMLSchema-instance"
    gn_brackets = "{"+generalname+"}"

    # registering namespaces 
    ET.register_namespace('',generalname)
    ET.register_namespace('xsi',xsi_name)

    #parsing xml file to python and get the root of the existing xml file
    RTCruntime = ET.parse('rtc_template_files\\rtcRuntimeConfig_empty.xml')
    myroot = RTCruntime.getroot()

    #convert date and runtime to required input for runtimeconfig file
    import datetime
    Starttime = datetime.timedelta(seconds=dfmmodel.mdu_parameters['tstart'])
    Timediff = datetime.timedelta(seconds=dfmmodel.mdu_parameters['tstop'])

    from datetime import datetime
    dt = str(dfmmodel.mdu_parameters['refdate'])+ ' ' + str(Starttime)
    startdatetime = datetime.strptime(dt,'%Y%m%d %H:%M:%S')

    stopdatetime = startdatetime + Timediff

    startd_str = str(startdatetime.date())
    startt_str = str(startdatetime.time())
    stopd_str = str(stopdatetime.date())
    stopt_str = str(stopdatetime.time())

    # replace start/stop dates and times in xml file 
    for x in myroot.iter(gn_brackets+"startDate"):
        x.set('date', startd_str)
        x.set('time', startt_str)        
    for x in myroot.iter(gn_brackets+"endDate"):
        x.set('date',stopd_str)
        x.set('time',stopt_str)  
    for x in myroot.iter(gn_brackets+"timeStep"):
        x.set('unit','second') 
        x.set('divider', '1')
        x.set('multiplier',str(rtc_update_int)) 

    xmlpath = os.path.join(outputdir,"rtc//rtcRuntimeConfig.xml")
    print("xml will be written to " + xmlpath)
    #write new xml file 
    RTCruntime.write(xmlpath)

    #opening xml file to implement a declaration (first line in xml file)
    xml = (bytes('<?xml version="1.0" encoding="utf-8" standalone="yes"?>\n', encoding='utf-8') + ET.tostring(myroot))
    xml = xml.decode('utf-8')
    with open(xmlpath, 'w+') as f:
        f.write(xml)

xml will be written to D:\GITHUB\PILOT_HENA\modelbouwscript\dellen\rtc//rtcRuntimeConfig.xml


#### Writing timeseries_import.xml file 

In [80]:
# Writing timeseries_import file (only if interval rule or time rule is active)

if RTC == 1 and (intervalrule == 1 or timeseriesrule == 1):
    
    #namespaces for this xml file
    generalnameT = "http://www.wldelft.nl/fews/PI"
    xsi_name = "http://www.w3.org/2001/XMLSchema-instance"
    gn_bracketsT = "{"+generalnameT+"}"

    # registering namespaces 
    ET.register_namespace('',generalnameT)
    ET.register_namespace('xsi',xsi_name)

    #parsing xml file to python and get the root of the existing xml file
    RTCTimeSeries = ET.parse('rtc_template_files\\timeseries_import_empty.xml')
    myroot = RTCTimeSeries.getroot()
    
    print("timeseries input is located in file " + str(timeseriesfile))
    
    if intervalrule == 1: 
        for x in range(len(interval_pumps)):
            a = ET.SubElement(myroot,gn_bracketsT+"series")
            a.tail = "\n  "    
            a.text = "\n    "
            myroot.tail = "\n"
            myroot.text = "\n  " 

            #header
            b = ET.SubElement(a,gn_bracketsT+"header")
            b.tail = "\n    "  
            b.text = "\n      "  

            #standard settings
            c = ET.SubElement(b,gn_bracketsT+"type")
            c.text = "instantaneous"
            c.tail = "\n      "

            d = ET.SubElement(b,gn_bracketsT+"locationId")
            d.text = "[IntervalRule]"+controlgroups_int_pumps[x]+"/Interval Rule"
            d.tail = "\n      "

            e = ET.SubElement(b,gn_bracketsT+"parameterId")
            e.text = "SP"
            e.tail = "\n      "

            f = ET.SubElement(b,gn_bracketsT+"timeStep")
            f.set("unit", "second")
            f.set("multiplier", str(rtc_update_int))
            f.set("divider", "1")
            f.tail = "\n      "

            # timeseries from file
            ts_file = pd.read_excel(timeseriesfile, sheet_name=interval_pumps[x])        

            g = ET.SubElement(b,gn_bracketsT+"startDate")
            g.set("date", str(ts_file.Datetime[0]).split(" ")[0])
            g.set("time", str(ts_file.Datetime[0]).split(" ")[1])
            g.tail = "\n      "

            h = ET.SubElement(b,gn_bracketsT+"endDate")
            h.set("date", str(ts_file.Datetime.iloc[1]).split(" ")[0])   #siebe: index -1 vervangen door 1
            h.set("time", str(ts_file.Datetime.iloc[1]).split(" ")[1])   #siebe: index -1 vervangen door 1
            h.tail = "\n      "

            i = ET.SubElement(b,gn_bracketsT+"missVal")
            i.text = "-999.0"
            i.tail = "\n      "

            j = ET.SubElement(b,gn_bracketsT+"stationName")
            j.tail = "\n      "

            k = ET.SubElement(b,gn_bracketsT+"units")
            k.tail = "\n    "

            for i in range(len(ts_file)):
                b = ET.SubElement(a,gn_bracketsT+"event")
                b.tail = "\n    "  
                if i == len(ts_file)-1:
                    b.tail = "\n  "
                b.set("date", str(ts_file.Datetime[i]).split(" ")[0])
                b.set("time", str(ts_file.Datetime[i]).split(" ")[1])
                b.set("value",str(ts_file.value[i]))
        
        for x in range(len(interval_weirs)):
            a = ET.SubElement(myroot,gn_bracketsT+"series")
            a.tail = "\n  "  
            if x == len(interval_weirs)-1 and timeseriesrule == 0:
                a.tail = "\n"  
            a.text = "\n    "
            myroot.tail = "\n"
            myroot.text = "\n  " 

            #header
            b = ET.SubElement(a,gn_bracketsT+"header")
            b.tail = "\n    "  
            b.text = "\n      "  

            #standard settings
            c = ET.SubElement(b,gn_bracketsT+"type")
            c.text = "instantaneous"
            c.tail = "\n      "

            d = ET.SubElement(b,gn_bracketsT+"locationId")
            d.text = "[IntervalRule]"+controlgroups_int[x]+"/Interval Rule"
            d.tail = "\n      "

            e = ET.SubElement(b,gn_bracketsT+"parameterId")
            e.text = "SP"
            e.tail = "\n      "

            f = ET.SubElement(b,gn_bracketsT+"timeStep")
            f.set("unit", "second")
            f.set("multiplier", str(rtc_update_int))
            f.set("divider", "1")
            f.tail = "\n      "
            
            # timeseries from file
            ts_file = pd.read_excel(timeseriesfile, sheet_name=interval_weirs[x])        

            g = ET.SubElement(b,gn_bracketsT+"startDate")
            g.set("date", str(ts_file.Datetime[0]).split(" ")[0])
            g.set("time", str(ts_file.Datetime[0]).split(" ")[1])
            g.tail = "\n      "

            h = ET.SubElement(b,gn_bracketsT+"endDate")
            h.set("date", str(ts_file.Datetime.iloc[1]).split(" ")[0])   #siebe: index -1 vervangen door 1
            h.set("time", str(ts_file.Datetime.iloc[1]).split(" ")[1])   #siebe: index -1 vervangen door 1
            h.tail = "\n      "

            i = ET.SubElement(b,gn_bracketsT+"missVal")
            i.text = "-999.0"
            i.tail = "\n      "

            j = ET.SubElement(b,gn_bracketsT+"stationName")
            j.tail = "\n      "

            k = ET.SubElement(b,gn_bracketsT+"units")
            k.tail = "\n    "

            for i in range(len(ts_file)):
                b = ET.SubElement(a,gn_bracketsT+"event")
                b.tail = "\n    "  
                if i == len(ts_file)-1:
                    b.tail = "\n  "
                b.set("date", str(ts_file.Datetime[i]).split(" ")[0])
                b.set("time", str(ts_file.Datetime[i]).split(" ")[1])
                b.set("value",str(ts_file.value[i]))
    
    if timeseriesrule == 1: 
        for x in range(len(timeserie_pumps)):
            a = ET.SubElement(myroot,gn_bracketsT+"series")
            a.tail = "\n  "  
            if x == len(timeserie_pumps)-1:
                a.tail = "\n"  
            a.text = "\n    "
            myroot.tail = "\n"
            myroot.text = "\n  " 

            #header
            b = ET.SubElement(a,gn_bracketsT+"header")
            b.tail = "\n    "  
            b.text = "\n      "  

            #standard settings
            c = ET.SubElement(b,gn_bracketsT+"type")
            c.text = "instantaneous"
            c.tail = "\n      "

            d = ET.SubElement(b,gn_bracketsT+"locationId")
            d.text = "[TimeRule]"+controlgroups_time_pumps[x]+"/Time Rule"
            d.tail = "\n      "

            e = ET.SubElement(b,gn_bracketsT+"parameterId")
            e.text = "TimeSeries"
            e.tail = "\n      "

            f = ET.SubElement(b,gn_bracketsT+"timeStep")
            f.set("unit", "second")
            f.set("multiplier", str(rtc_update_int))
            f.set("divider", "1")
            f.tail = "\n      "

            # timeseries from file 
            ts_file = pd.read_excel(timeseriesfile, sheet_name=timeserie_pumps[x])
            
            g = ET.SubElement(b,gn_bracketsT+"startDate")
            g.set("date", str(ts_file.Datetime[0]).split(" ")[0])
            g.set("time", str(ts_file.Datetime[0]).split(" ")[1])
            g.tail = "\n      "

            h = ET.SubElement(b,gn_bracketsT+"endDate")
            h.set("date", str(ts_file.Datetime.iloc[1]).split(" ")[0])   #siebe: index -1 vervangen door 1
            h.set("time", str(ts_file.Datetime.iloc[1]).split(" ")[1])   #siebe: index -1 vervangen door 1
            h.tail = "\n      "

            i = ET.SubElement(b,gn_bracketsT+"missVal")
            i.text = "-999.0"
            i.tail = "\n      "

            j = ET.SubElement(b,gn_bracketsT+"stationName")
            j.tail = "\n      "

            k = ET.SubElement(b,gn_bracketsT+"units")
            k.tail = "\n    "

            for i in range(len(ts_file)):
                b = ET.SubElement(a,gn_bracketsT+"event")
                b.tail = "\n    "  
                if i == len(ts_file)-1:
                    b.tail = "\n  "
                b.set("date", str(ts_file.Datetime[i]).split(" ")[0])
                b.set("time", str(ts_file.Datetime[i]).split(" ")[1])
                b.set("value",str(ts_file.value[i]))   
        
        for x in range(len(timeserie_weirs)):
            
            print("creating timeseries for weir " + str(timeserie_weirs[x]))
            
            a = ET.SubElement(myroot,gn_bracketsT+"series")
            a.tail = "\n  "  
            if x == len(timeserie_weirs)-1:
                a.tail = "\n"  
            a.text = "\n    "
            myroot.tail = "\n"
            myroot.text = "\n  " 

            #header
            b = ET.SubElement(a,gn_bracketsT+"header")
            b.tail = "\n    "  
            b.text = "\n      "  

            #standard settings
            c = ET.SubElement(b,gn_bracketsT+"type")
            c.text = "instantaneous"
            c.tail = "\n      "

            d = ET.SubElement(b,gn_bracketsT+"locationId")
            d.text = "[TimeRule]"+controlgroups_time[x]+"/Time Rule"
            d.tail = "\n      "

            e = ET.SubElement(b,gn_bracketsT+"parameterId")
            e.text = "TimeSeries"
            e.tail = "\n      "

            f = ET.SubElement(b,gn_bracketsT+"timeStep")
            f.set("unit", "second")
            f.set("multiplier", str(rtc_update_int))
            f.set("divider", "1")
            f.tail = "\n      "

            # timeseries from file 
            ts_file = pd.read_excel(timeseriesfile, sheet_name=timeserie_weirs[x])
            
            g = ET.SubElement(b,gn_bracketsT+"startDate")
            g.set("date", str(ts_file.Datetime[0]).split(" ")[0])
            g.set("time", str(ts_file.Datetime[0]).split(" ")[1])
            g.tail = "\n      "

            print("startdate is " + str(ts_file.Datetime[0]))
            print("end date is " + str(ts_file.Datetime[1]))
            
            h = ET.SubElement(b,gn_bracketsT+"endDate")
            h.set("date", str(ts_file.Datetime.iloc[1]).split(" ")[0])   #siebe: index -1 vervangen door 1
            h.set("time", str(ts_file.Datetime.iloc[1]).split(" ")[1])   #siebe: index -1 vervangen door 1
            h.tail = "\n      "

            i = ET.SubElement(b,gn_bracketsT+"missVal")
            i.text = "-999.0"
            i.tail = "\n      "

            j = ET.SubElement(b,gn_bracketsT+"stationName")
            j.tail = "\n      "

            k = ET.SubElement(b,gn_bracketsT+"units")
            k.tail = "\n    "

            for i in range(len(ts_file)):
                
                print("walking through ts_file with length " + str(len(ts_file)))
                print("item is " + str(ts_file.Datetime[i]))
                
                b = ET.SubElement(a,gn_bracketsT+"event")
                b.tail = "\n    "  
                if i == len(ts_file)-1:
                    b.tail = "\n  "
                b.set("date", str(ts_file.Datetime[i]).split(" ")[0])
                b.set("time", str(ts_file.Datetime[i]).split(" ")[1])
                b.set("value",str(ts_file.value[i]))       

xmlpath = os.path.join(outputdir,"rtc//timeseries_import.xml")
print("timeseries xml will be written to " + xmlpath)

#write new xml file 
RTCTimeSeries.write(xmlpath)

#opening xml file to implement a declaration (first line in xml file)
xml = (bytes('<?xml version="1.0" encoding="utf-8" standalone="yes"?>\n', encoding='utf-8') + ET.tostring(myroot))
xml = xml.decode('utf-8')
with open(xmlpath, 'w+') as f:
    f.write(xml)

timeseries input is located in file D:\GITHUB\PILOT_HENA\modelbouwscript\data\xlsx\timeseries_input.xlsx
creating timeseries for weir KST-O-17700
startdate is 2010-08-01 00:00:00
end date is 2010-08-01 10:00:00
walking through ts_file with length 6
item is 2010-08-01 00:00:00
walking through ts_file with length 6
item is 2010-08-01 10:00:00
walking through ts_file with length 6
item is 2010-08-01 20:00:00
walking through ts_file with length 6
item is 2010-08-01 22:00:00
walking through ts_file with length 6
item is 2010-08-20 00:00:00
walking through ts_file with length 6
item is 2010-08-25 00:00:00
creating timeseries for weir KST-O-17800
startdate is 2010-08-01 00:00:00
end date is 2010-08-01 10:00:00
walking through ts_file with length 6
item is 2010-08-01 00:00:00
walking through ts_file with length 6
item is 2010-08-01 10:00:00
walking through ts_file with length 6
item is 2010-08-01 20:00:00
walking through ts_file with length 6
item is 2010-08-01 22:00:00
walking through ts_file

#### Writing rtcToolsConfig.xml file 

In [81]:
if RTC == 1:
    
    generalnameT = "http://www.wldelft.nl/fews"
    xsi_name = "http://www.w3.org/2001/XMLSchema-instance"
    gn_brackets = "{"+generalnameT+"}"

    # registering namespaces 
    ET.register_namespace('',generalname)
    ET.register_namespace('xsi',xsi_name)

    #parsing xml file
    RTCtoolsconfig = ET.parse('rtc_template_files\\rtcToolsConfig_empty.xml')
    myroot = RTCtoolsconfig.getroot()
   
    # writing a rule "pid controller" for every weir in the dataset (the .tail and .text lines are just give the indents,
    # so the file is readable.  
    if pidrule == 1: 
        for x in range(len(weirs_auto)):            
            a = ET.SubElement(myroot[1],gn_brackets+"rule")
            a.tail = "\n    "  
            if x == len(weirs_auto)-1 and timeseriesrule == 0 and intervalrule == 0:
                a.tail = "\n  "  
            a.text = "\n      "
            myroot[1].tail = "\n"
            myroot[1].text = "\n    "
            
            #rule type (PID)
            b = ET.SubElement(a,gn_brackets+"pid")
            b.tail = "\n    "  
            b.text = "\n        "  
            b.set("id", "[PID]"+str(controlgroups[x])+"/PID Rule")
            
            #standard settings
            c = ET.SubElement(b,gn_brackets+"mode")
            c.text = "PIDVEL"
            c.tail = "\n        "
            
            d = ET.SubElement(b,gn_brackets+"settingMin")
            d.tail = "\n        "
            d.text = str(settingMin[x])
            
            e = ET.SubElement(b,gn_brackets+"settingMax")
            e.tail = "\n        "
            e.text = str(settingMax[x])
            
            f = ET.SubElement(b,gn_brackets+"settingMaxSpeed")
            f.tail = "\n        "
            f.text = str(settingMaxspeed)
            
            g = ET.SubElement(b,gn_brackets+"kp")
            g.tail = "\n        "
            g.text = str(kp)
            
            h = ET.SubElement(b,gn_brackets+"ki")
            h.tail = "\n        "
            h.text = str(ki)
            
            i = ET.SubElement(b,gn_brackets+"kd")
            i.tail = "\n        "
            i.text = str(kd)
            
            # input
            j = ET.SubElement(b,gn_brackets+"input")
            j.tail = "\n        "
            j.text = "\n          "
            
            k = ET.SubElement(j,gn_brackets+"x")
            k.tail = "\n          "
            k.text = "[Input]"+observationpoints[x]+"/Water level (op)"
            
            l = ET.SubElement(j,gn_brackets+"setpointValue")
            l.tail = "\n        "
            l.text = str(controllevel[x])
            
            #output
            m = ET.SubElement(b,gn_brackets+"output")
            m.tail = "\n      "
            m.text = "\n          "
            
            o = ET.SubElement(m,gn_brackets+"y")
            o.tail = "\n          "
            o.text = "[Output]"+str(weirs_auto[x])+"/Crest level (s)"
            
            p = ET.SubElement(m,gn_brackets+"integralPart")
            p.tail = "\n          "
            p.text = "[IP]"+controlgroups[x]+"/PID Rule"
            
            q = ET.SubElement(m,gn_brackets+"differentialPart")
            q.tail = "\n        "
            q.text = "[DP]"+controlgroups[x]+"/PID Rule"

    #writing the root to a new xml file
    xmlpath = os.path.join(outputdir,"rtc//rtcToolsConfig.xml")
    print("RtcTools Config xml will be written to " + xmlpath)
    RTCtoolsconfig.write(xmlpath)

    #opening xml file to implement a declaration (first line in xml file)
    xml = (bytes('<?xml version="1.0" encoding="utf-8" standalone="yes"?>\n', encoding='utf-8') + ET.tostring(myroot))
    xml = xml.decode('utf-8')
    with open(xmlpath, 'w+') as f:
        f.write(xml)            
            
    if intervalrule == 1: 
        for x in range(len(interval_pumps)):
            a = ET.SubElement(myroot[1],gn_brackets+"rule")
            a.tail = "\n    "  
            a.text = "\n      "
            myroot[1].tail = "\n"
            myroot[1].text = "\n    "
            
            #rule type (inteval)
            b = ET.SubElement(a,gn_brackets+"interval")
            b.tail = "\n    "  
            b.text = "\n        "  
            b.set("id", "[IntervalRule]"+str(controlgroups_int_pumps[x])+"/Interval Rule")
            
            #standard settings
            d = ET.SubElement(b,gn_brackets+"settingBelow")
            d.tail = "\n        "
            d.text = str(settingBelow_pump[x])
            
            e = ET.SubElement(b,gn_brackets+"settingAbove")
            e.tail = "\n        "
            e.text = str(settingAbove_pump[x])
            
            # dit nog uitzoeken!!!!!!! nu alleen variable nog gebruiken
            if pd.read_excel(timeseriesfile, sheet_name=interval_pumps[x]).intervaltype[0] == "FIXED":
                f = ET.SubElement(b,gn_brackets+"settingMaxStep")
                f.tail = "\n        "
                f.text = str(settingMaxstep)
                
            elif pd.read_excel(timeseriesfile, sheet_name=interval_pumps[x]).intervaltype[0] == "VARIABLE":
                f = ET.SubElement(b,gn_brackets+"settingMaxSpeed")
                f.tail = "\n        "
                f.text = str(settingMaxspeed_int_pumps)
            
            g = ET.SubElement(b,gn_brackets+"deadbandSetpointAbsolute")
            g.tail = "\n        "
            g.text = str(deadbandSetpointAbsolute)
            
            #input
            j = ET.SubElement(b,gn_brackets+"input")
            j.tail = "\n        "
            j.text = "\n          "         
            
            k = ET.SubElement(j,gn_brackets+"x")
            k.tail = "\n          "
            k.text = "[Input]"+observationpoints_int_pumps[x]+"/Water level (op)"
            k.set("ref", "EXPLICIT")
            
            l = ET.SubElement(j,gn_brackets+"setpoint")
            l.tail = "\n        "
            l.text = "[SP]"+controlgroups_int_pumps[x]+ "/Interval Rule"
            
            #output
            m = ET.SubElement(b,gn_brackets+"output")
            m.tail = "\n      "
            m.text = "\n          "
            
            o = ET.SubElement(m,gn_brackets+"y")
            o.tail = "\n          "
            o.text = "[Output]"+str(interval_pumps[x])+"/Setpoint (s)"
            
            p = ET.SubElement(m,gn_brackets+"status")
            p.tail = "\n          "
            p.text = "[Status]"+controlgroups_int_pumps[x]+"/Interval Rule"
        
        for x in range(len(interval_weirs)):
            a = ET.SubElement(myroot[1],gn_brackets+"rule")
            a.tail = "\n    "  
            if x == len(interval_weirs)-1 and timeseriesrule == 0:
                a.tail = "\n  "  
            a.text = "\n      "
            myroot[1].tail = "\n"
            myroot[1].text = "\n    "
            
            #rule type (inteval)
            b = ET.SubElement(a,gn_brackets+"interval")
            b.tail = "\n    "  
            b.text = "\n        "  
            b.set("id", "[IntervalRule]"+str(controlgroups_int[x])+"/Interval Rule")
            
            #standard settings
            d = ET.SubElement(b,gn_brackets+"settingBelow")
            d.tail = "\n        "
            d.text = str(settingBelow[x])
            
            e = ET.SubElement(b,gn_brackets+"settingAbove")
            e.tail = "\n        "
            e.text = str(settingAbove[x])
            
            # dit nog uitzoeken!!!!!!! nu alleen variable nog gebruiken
            if pd.read_excel(timeseriesfile, sheet_name=interval_weirs[x]).intervaltype[0] == "FIXED":
                f = ET.SubElement(b,gn_brackets+"settingMaxStep")
                f.tail = "\n        "
                f.text = str(settingMaxstep)
                
            elif pd.read_excel(timeseriesfile, sheet_name=interval_weirs[x]).intervaltype[0] == "VARIABLE":
                f = ET.SubElement(b,gn_brackets+"settingMaxSpeed")
                f.tail = "\n        "
                f.text = str(settingMaxspeed_int)
            
            g = ET.SubElement(b,gn_brackets+"deadbandSetpointAbsolute")
            g.tail = "\n        "
            g.text = str(deadbandSetpointAbsolute)
            
            #input
            j = ET.SubElement(b,gn_brackets+"input")
            j.tail = "\n        "
            j.text = "\n          "         
            
            k = ET.SubElement(j,gn_brackets+"x")
            k.tail = "\n          "
            k.text = "[Input]"+observationpoints_int[x]+"/Water level (op)"
            k.set("ref", "EXPLICIT")
            
            l = ET.SubElement(j,gn_brackets+"setpoint")
            l.tail = "\n        "
            l.text = "[SP]"+controlgroups_int[x]+ "/Interval Rule"
            
            #output
            m = ET.SubElement(b,gn_brackets+"output")
            m.tail = "\n      "
            m.text = "\n          "
            
            o = ET.SubElement(m,gn_brackets+"y")
            o.tail = "\n          "
            o.text = "[Output]"+str(interval_weirs[x])+"/Crest level (s)"
            
            p = ET.SubElement(m,gn_brackets+"status")
            p.tail = "\n          "
            p.text = "[Status]"+controlgroups_int[x]+"/Interval Rule"
    
    if timeseriesrule == 1: 
        for x in range(len(timeserie_pumps)):
            a = ET.SubElement(myroot[1],gn_brackets+"rule")
            a.tail = "\n    "  
            if x == len(timeserie_pumps)-1:
                a.tail = "\n  "  
            a.text = "\n      "
            myroot[1].tail = "\n"
            myroot[1].text = "\n    "
            
            # rule type (timerule)
            b = ET.SubElement(a,gn_brackets+"timeAbsolute")
            b.tail = "\n    "  
            b.text = "\n        "  
            b.set("id", "[TimeRule]"+str(controlgroups_time_pumps[x])+"/Time Rule")
            
            # input
            j = ET.SubElement(b,gn_brackets+"input")
            j.tail = "\n        "
            j.text = "\n          "
            
            k = ET.SubElement(j,gn_brackets+"x")
            k.tail = "\n        "
            k.text = str(controlgroups_time_pumps[x])+"/Time Rule"
            
            #output
            m = ET.SubElement(b,gn_brackets+"output")
            m.tail = "\n        "
            m.text = "\n          "
            
            o = ET.SubElement(m,gn_brackets+"y")
            o.tail = "\n        "        
            o.text = "[Output]"+str(timeserie_pumps[x])+"/Setpoint (s)"
        
        for x in range(len(timeserie_weirs)):
            a = ET.SubElement(myroot[1],gn_brackets+"rule")
            a.tail = "\n    "  
            if x == len(timeserie_weirs)-1:
                a.tail = "\n  "  
            a.text = "\n      "
            myroot[1].tail = "\n"
            myroot[1].text = "\n    "
            
            # rule type (timerule)
            b = ET.SubElement(a,gn_brackets+"timeAbsolute")
            b.tail = "\n    "  
            b.text = "\n        "  
            b.set("id", "[TimeRule]"+str(controlgroups_time[x])+"/Time Rule")
            
            # input
            j = ET.SubElement(b,gn_brackets+"input")
            j.tail = "\n        "
            j.text = "\n          "
            
            k = ET.SubElement(j,gn_brackets+"x")
            k.tail = "\n        "
            k.text = str(controlgroups_time[x])+"/Time Rule"
            
            #output
            m = ET.SubElement(b,gn_brackets+"output")
            m.tail = "\n        "
            m.text = "\n          "
            
            o = ET.SubElement(m,gn_brackets+"y")
            o.tail = "\n        "        
            o.text = "[Output]"+str(timeserie_weirs[x])+"/Crest level (s)"

    #writing the root to a new xml file
    RTCtoolsconfig.write(xmlpath)

    #opening xml file to implement a declaration (first line in xml file)
    xml = (bytes('<?xml version="1.0" encoding="utf-8" standalone="yes"?>\n', encoding='utf-8') + ET.tostring(myroot))
    xml = xml.decode('utf-8')
    with open(xmlpath, 'w+') as f:
        f.write(xml)
        
print("RtcToolsConfig has been written.")

RtcTools Config xml will be written to D:\GITHUB\PILOT_HENA\modelbouwscript\dellen\rtc//rtcToolsConfig.xml
RtcToolsConfig has been written.


#### Writing dataconfig.xml file 

In [82]:
if RTC == 1:
    # registering namespaces 
    ET.register_namespace('',generalname)
    ET.register_namespace('xsi',xsi_name)

    # Parsing xml file
    RTCdataconfig = ET.parse('rtc_template_files\\rtcDataConfig_empty.xml')
    myroot = RTCdataconfig.getroot()

    # implementing standard settings import and exportdata
    a0 = ET.SubElement(myroot[1],gn_brackets+"CSVTimeSeriesFile")
    a0.set("decimalSeparator", ".")
    a0.set("delimiter",",")
    a0.set("adjointOutput", "false")
    a0.tail = "\n    "  
    a0.text = ""  
    
    a1 = ET.SubElement(myroot[1],gn_brackets+"PITimeSeriesFile")     
    a1.tail = "\n     "  
    a1.text = "\n      "  
    
    a2 = ET.SubElement(a1,gn_brackets+"timeSeriesFile")
    a2.text = "timeseries_export.xml"     
    a2.tail = "\n      "  
    
    a3 = ET.SubElement(a1,gn_brackets+"useBinFile")    
    a3.text = "false"     
    a3.tail = "\n    "  
    
    if timeseriesrule == 1 or intervalrule == 1 :
        a4 = ET.SubElement(myroot[0],gn_brackets+"PITimeSeriesFile")     
        a4.tail = "\n    "  
        a4.text = "\n      "  

        a5 = ET.SubElement(a4,gn_brackets+"timeSeriesFile")
        a5.text = "timeseries_import.xml"     
        a5.tail = "\n      "  

        a6 = ET.SubElement(a4,gn_brackets+"useBinFile")    
        a6.text = "false"     
        a6.tail = "\n    " 

    #weir dependable data
    if pidrule == 1: 
        for x in range(len(weirs_auto)):

            # te importeren data
            a = ET.SubElement(myroot[0],gn_brackets+"timeSeries")
            a.tail = "\n    "  
            if x == len(weirs_auto)-1 and timeseriesrule == 0 and intervalrule == 0:
                a.tail = "\n  " 
            a.text = "\n      "
            myroot[0].text = "\n    "
            a.set("id", "[Input]"+observationpoints[x]+"/Water level (op)")

            b = ET.SubElement(a,gn_brackets+"OpenMIExchangeItem")
            b.tail = "\n    "  
            b.text = "\n        "  

            c = ET.SubElement(b,gn_brackets+"elementId")
            c.text = observationpoints[x]
            c.tail = "\n        "

            d = ET.SubElement(b,gn_brackets+"quantityId")
            d.text = "Water level (op)"
            d.tail = "\n        "

            e = ET.SubElement(b,gn_brackets+"unit")
            e.text = "m"
            e.tail = "\n      "

            # te exporteren data: 
            f = ET.SubElement(myroot[1],gn_brackets+"timeSeries")
            f.tail = "\n    "  
            if x == len(weirs_auto)-1 and timeseriesrule == 0 and intervalrule == 0:
                f.tail = "\n    " 
            f.text = "\n      "
            myroot[1][1].tail = "\n    "
            f.set("id", "[Output]"+str(weirs_auto[x])+"/Crest level (s)")

            g = ET.SubElement(f,gn_brackets+"OpenMIExchangeItem")
            g.tail = "\n    "  
            g.text = "\n        "  

            h = ET.SubElement(g,gn_brackets+"elementId")
            h.text = str(weirs_auto[x])
            h.tail = "\n        "

            j = ET.SubElement(g,gn_brackets+"quantityId")
            j.text = "Crest level (s)"
            j.tail = "\n        "

            k = ET.SubElement(g,gn_brackets+"unit")
            k.text = "m"
            k.tail = "\n      "
            
        for x in range(len(weirs_auto)):   
            i = ET.SubElement(myroot[1],gn_brackets+"timeSeries")
            i.set("id", "[IP]"+str(controlgroups[x])+"/PID Rule")
            i.tail = "\n    "

            j = ET.SubElement(myroot[1],gn_brackets+"timeSeries")
            j.set("id", "[DP]"+str(controlgroups[x])+"/PID Rule")
            j.tail = "\n    "
            if x == len(weirs_auto)-1 and intervalrule == 0 and timeseriesrule == 0:
                j.tail = "\n  "
    
    if intervalrule == 1: 
        for x in range(len(interval_pumps)):     
            
            # te importeren data
            a = ET.SubElement(myroot[0],gn_brackets+"timeSeries")
            a.tail = "\n    "  
            a.text = "\n      "
            myroot[0].text = "\n    "
            a.set("id", "[SP]"+controlgroups_int_pumps[x]+"/Interval Rule")
            
            b = ET.SubElement(a,gn_brackets+"PITimeSeries")
            b.tail = "\n    "  
            b.text = "\n        "  

            c = ET.SubElement(b,gn_brackets+"locationId")
            c.text = "[IntervalRule]"+controlgroups_int_pumps[x]+"/Interval Rule"
            c.tail = "\n        "

            d = ET.SubElement(b,gn_brackets+"parameterId")
            d.text = "SP"
            d.tail = "\n        "

            e = ET.SubElement(b,gn_brackets+"interpolationOption")
            e.text = str(pd.read_excel(timeseriesfile, sheet_name=interval_pumps[x]).interpolation[0])
            e.tail = "\n        "
            
            f = ET.SubElement(b,gn_brackets+"extrapolationOption")
            f.text = str(pd.read_excel(timeseriesfile, sheet_name=interval_pumps[x]).extrapolation[0])
            f.tail = "\n      "
            
            g = ET.SubElement(myroot[0],gn_brackets+"timeSeries")
            g.tail = "\n    "  
            g.text = "\n      "
            myroot[0].text = "\n    "
            g.set("id", "[Input]"+observationpoints_int_pumps[x]+"/Water level (op)")

            h = ET.SubElement(g,gn_brackets+"OpenMIExchangeItem")
            h.tail = "\n    "  
            h.text = "\n        "  

            i = ET.SubElement(h,gn_brackets+"elementId")
            i.text = observationpoints_int_pumps[x]
            i.tail = "\n        "

            j = ET.SubElement(h,gn_brackets+"quantityId")
            j.text = "Water level (op)"
            j.tail = "\n        "

            k = ET.SubElement(h,gn_brackets+"unit")
            k.text = "m"
            k.tail = "\n      "
            
            # te exporteren data: 
            f = ET.SubElement(myroot[1],gn_brackets+"timeSeries")
            f.tail = "\n    "  
            f.text = "\n      "
            myroot[1][1].tail = "\n    "
            f.set("id", "[Output]"+str(interval_pumps[x])+"/Setpoint (s)")

            g = ET.SubElement(f,gn_brackets+"OpenMIExchangeItem")
            g.tail = "\n    "  
            g.text = "\n        "  

            h = ET.SubElement(g,gn_brackets+"elementId")
            h.text = str(interval_pumps[x])
            h.tail = "\n        "

            j = ET.SubElement(g,gn_brackets+"quantityId")
            j.text = "Setpoint (s)"
            j.tail = "\n        "

            k = ET.SubElement(g,gn_brackets+"unit")
            k.text = "m"
            k.tail = "\n      "
            
        for x in range(len(interval_pumps)):      
            i = ET.SubElement(myroot[1],gn_brackets+"timeSeries")
            i.set("id", "[Status]"+str(controlgroups_int_pumps[x])+"/Interval Rule")
            i.tail = "\n    "
        
        for x in range(len(interval_weirs)):     
            
            # te importeren data
            a = ET.SubElement(myroot[0],gn_brackets+"timeSeries")
            a.tail = "\n    "  
            if x == len(interval_weirs)-1 and timeseriesrule == 0: 
                a.tail = "\n  " 
            a.text = "\n      "
            myroot[0].text = "\n    "
            a.set("id", "[SP]"+controlgroups_int[x]+"/Interval Rule")
            
            b = ET.SubElement(a,gn_brackets+"PITimeSeries")
            b.tail = "\n    "  
            b.text = "\n        "  

            c = ET.SubElement(b,gn_brackets+"locationId")
            c.text = "[IntervalRule]"+controlgroups_int[x]+"/Interval Rule"
            c.tail = "\n        "

            d = ET.SubElement(b,gn_brackets+"parameterId")
            d.text = "SP"
            d.tail = "\n        "

            e = ET.SubElement(b,gn_brackets+"interpolationOption")
            e.text = str(pd.read_excel(timeseriesfile, sheet_name=interval_weirs[x]).interpolation[0])
            e.tail = "\n        "
            
            f = ET.SubElement(b,gn_brackets+"extrapolationOption")
            f.text = str(pd.read_excel(timeseriesfile, sheet_name=interval_weirs[x]).extrapolation[0])
            f.tail = "\n      "
            
            g = ET.SubElement(myroot[0],gn_brackets+"timeSeries")
            g.tail = "\n    "  
            if x == len(interval_weirs)-1 and timeseriesrule == 0:
                g.tail = "\n  " 
            g.text = "\n      "
            myroot[0].text = "\n    "
            g.set("id", "[Input]"+observationpoints_int[x]+"/Water level (op)")

            h = ET.SubElement(g,gn_brackets+"OpenMIExchangeItem")
            h.tail = "\n    "  
            h.text = "\n        "  

            i = ET.SubElement(h,gn_brackets+"elementId")
            i.text = observationpoints_int[x]
            i.tail = "\n        "

            j = ET.SubElement(h,gn_brackets+"quantityId")
            j.text = "Water level (op)"
            j.tail = "\n        "

            k = ET.SubElement(h,gn_brackets+"unit")
            k.text = "m"
            k.tail = "\n      "
            
            # te exporteren data: 
            f = ET.SubElement(myroot[1],gn_brackets+"timeSeries")
            f.tail = "\n    "  
            if x == len(interval_weirs)-1 and timeseriesrule == 0: 
                f.tail = "\n    " 
            f.text = "\n      "
            myroot[1][1].tail = "\n    "
            f.set("id", "[Output]"+str(interval_weirs[x])+"/Crest level (s)")

            g = ET.SubElement(f,gn_brackets+"OpenMIExchangeItem")
            g.tail = "\n    "  
            g.text = "\n        "  

            h = ET.SubElement(g,gn_brackets+"elementId")
            h.text = str(interval_weirs[x])
            h.tail = "\n        "

            j = ET.SubElement(g,gn_brackets+"quantityId")
            j.text = "Crest level (s)"
            j.tail = "\n        "

            k = ET.SubElement(g,gn_brackets+"unit")
            k.text = "m"
            k.tail = "\n      "
            
        for x in range(len(interval_weirs)):      
            i = ET.SubElement(myroot[1],gn_brackets+"timeSeries")
            i.set("id", "[Status]"+str(controlgroups_int[x])+"/Interval Rule")
            i.tail = "\n    "
            if x == len(interval_weirs)-1 and timeseriesrule == 0:
                i.tail = "\n  "
                
    if timeseriesrule == 1: 
        for x in range(len(timeserie_pumps)):     
            
            # te importeren data
            a = ET.SubElement(myroot[0],gn_brackets+"timeSeries")
            a.tail = "\n    "  
            if x == len(interval_pumps)-1: 
                a.tail = "\n  " 
            a.text = "\n      "
            myroot[0].text = "\n    "
            a.set("id", controlgroups_time_pumps[x]+"/Time Rule")
            
            b = ET.SubElement(a,gn_brackets+"PITimeSeries")
            b.tail = "\n    "  
            b.text = "\n        "  

            c = ET.SubElement(b,gn_brackets+"locationId")
            c.text = "[TimeRule]"+controlgroups_time_pumps[x]+"/Time Rule"
            c.tail = "\n        "

            d = ET.SubElement(b,gn_brackets+"parameterId")
            d.text = "TimeSeries"
            d.tail = "\n        "

            e = ET.SubElement(b,gn_brackets+"interpolationOption")
            e.text = str(pd.read_excel(timeseriesfile, sheet_name=timeserie_pumps[x]).interpolation[0])
            e.tail = "\n        "
            
            f = ET.SubElement(b,gn_brackets+"extrapolationOption")
            f.text = str(pd.read_excel(timeseriesfile, sheet_name=timeserie_pumps[x]).extrapolation[0])
            f.tail = "\n      "
            
            # te exporteren data: 
            f = ET.SubElement(myroot[1],gn_brackets+"timeSeries")
            f.tail = "\n    "  
            if x == len(timeserie_pumps)-1: 
                f.tail = "\n  " 
            f.text = "\n      "
            myroot[1][1].tail = "\n    "
            f.set("id", "[Output]"+str(timeserie_pumps[x])+"/Setpoint (s)")

            g = ET.SubElement(f,gn_brackets+"OpenMIExchangeItem")
            g.tail = "\n    "  
            g.text = "\n        "  

            h = ET.SubElement(g,gn_brackets+"elementId")
            h.text = str(timeserie_pumps[x])
            h.tail = "\n        "

            j = ET.SubElement(g,gn_brackets+"quantityId")
            j.text = "Setpoint (s)"
            j.tail = "\n        "

            k = ET.SubElement(g,gn_brackets+"unit")
            k.text = "m"
            k.tail = "\n      "
        
        for x in range(len(timeserie_weirs)):     
            
            # te importeren data
            a = ET.SubElement(myroot[0],gn_brackets+"timeSeries")
            a.tail = "\n    "  
            if x == len(interval_weirs)-1: 
                a.tail = "\n  " 
            a.text = "\n      "
            myroot[0].text = "\n    "
            a.set("id", controlgroups_time[x]+"/Time Rule")
            
            b = ET.SubElement(a,gn_brackets+"PITimeSeries")
            b.tail = "\n    "  
            b.text = "\n        "  

            c = ET.SubElement(b,gn_brackets+"locationId")
            c.text = "[TimeRule]"+controlgroups_time[x]+"/Time Rule"
            c.tail = "\n        "

            d = ET.SubElement(b,gn_brackets+"parameterId")
            d.text = "TimeSeries"
            d.tail = "\n        "

            e = ET.SubElement(b,gn_brackets+"interpolationOption")
            e.text = str(pd.read_excel(timeseriesfile, sheet_name=timeserie_weirs[x]).interpolation[0])
            e.tail = "\n        "
            
            f = ET.SubElement(b,gn_brackets+"extrapolationOption")
            f.text = str(pd.read_excel(timeseriesfile, sheet_name=timeserie_weirs[x]).extrapolation[0])
            f.tail = "\n      "
            
            # te exporteren data: 
            f = ET.SubElement(myroot[1],gn_brackets+"timeSeries")
            f.tail = "\n    "  
            if x == len(timeserie_weirs)-1: 
                f.tail = "\n  " 
            f.text = "\n      "
            myroot[1][1].tail = "\n    "
            f.set("id", "[Output]"+str(timeserie_weirs[x])+"/Crest level (s)")

            g = ET.SubElement(f,gn_brackets+"OpenMIExchangeItem")
            g.tail = "\n    "  
            g.text = "\n        "  

            h = ET.SubElement(g,gn_brackets+"elementId")
            h.text = str(timeserie_weirs[x])
            h.tail = "\n        "

            j = ET.SubElement(g,gn_brackets+"quantityId")
            j.text = "Crest level (s)"
            j.tail = "\n        "

            k = ET.SubElement(g,gn_brackets+"unit")
            k.text = "m"
            k.tail = "\n      "
    
    #writing the root to a new xml file
    xmlpath = os.path.join(outputdir,"rtc//rtcDataConfig.xml")
    print("RCT DataConfig xml will be written to " + xmlpath)
    RTCdataconfig.write(xmlpath)
    
    xml = (bytes('<?xml version="1.0" encoding="utf-8" standalone="yes"?>\n', encoding='utf-8') + ET.tostring(myroot))
    xml = xml.decode('utf-8')
    with open(xmlpath, 'w+') as f:
        f.write(xml)

RCT DataConfig xml will be written to D:\GITHUB\PILOT_HENA\modelbouwscript\dellen\rtc//rtcDataConfig.xml


#### Writing dimr-config.xml file using Elementree package.

In [83]:
if RTC == 1:
    # namespaces
    generalnamedimr = "http://schemas.deltares.nl/dimr" #Config
    gn_dimr_brackets = "{"+generalnamedimr+"}"

    # registering namespaces 
    ET.register_namespace('',generalnamedimr)
    ET.register_namespace('xsi',xsi_name)

    #parsing xml file
    RTCdimr_config = ET.parse('rtc_template_files\\dimr_config_empty.xml')
    myroot = RTCdimr_config.getroot()

    # finding the time input within the startgroup and replacing the numbers accordingly
    a = myroot[1][0][0][0]
    a.text = str(int(dfmmodel.mdu_parameters['tstart']))+" "+str(rtc_update_int)+" "+str(int(dfmmodel.mdu_parameters['tstop']))
    
    #finding mdu name and replacing it with mdu name
    b = myroot[3][2]
    b.text = str(name_mdu)+".mdu"
    
    if pidrule ==1: 
        for x in range(len(weirs_auto)):
            c = ET.SubElement(myroot[4],gn_dimr_brackets+"item")
            c.text = "\n      "
            c.tail = "\n  " 

            d = ET.SubElement(c,gn_dimr_brackets+"sourceName")
            d.text = "[Output]"+str(weirs_auto[x])+"/Crest level (s)"
            d.tail = "\n      "

            e = ET.SubElement(c,gn_dimr_brackets+"targetName")
            e.text = "weirs/"+str(weirs_auto[x])+"/CrestLevel"
            e.tail = "\n  "
    
    if intervalrule == 1:
        for x in range(len(interval_pumps)):
            c = ET.SubElement(myroot[4],gn_dimr_brackets+"item")
            c.text = "\n      "
            c.tail = "\n  " 

            d = ET.SubElement(c,gn_dimr_brackets+"sourceName")
            d.text = "[Output]"+str(interval_pumps[x])+"/Setpoint (s)"
            d.tail = "\n      "

            e = ET.SubElement(c,gn_dimr_brackets+"targetName")
            e.text = "pumps/"+str(interval_pumps[x])+"/capacity"
            e.tail = "\n  "
        
        for x in range(len(interval_weirs)):
            c = ET.SubElement(myroot[4],gn_dimr_brackets+"item")
            c.text = "\n      "
            c.tail = "\n  " 

            d = ET.SubElement(c,gn_dimr_brackets+"sourceName")
            d.text = "[Output]"+str(interval_weirs[x])+"/Crest level (s)"
            d.tail = "\n      "

            e = ET.SubElement(c,gn_dimr_brackets+"targetName")
            e.text = "weirs/"+str(interval_weirs[x])+"/CrestLevel"
            e.tail = "\n  "
    
    if timeseriesrule ==1:
        for x in range(len(timeserie_pumps)):  
            c = ET.SubElement(myroot[4],gn_dimr_brackets+"item")
            c.text = "\n      "
            c.tail = "\n  " 

            d = ET.SubElement(c,gn_dimr_brackets+"sourceName")
            d.text = "[Output]"+str(timeserie_pumps[x])+"/Setpoint (s)"
            d.tail = "\n      "

            e = ET.SubElement(c,gn_dimr_brackets+"targetName")
            e.text = "pumps/"+str(timeserie_pumps[x])+"/capacity"
            e.tail = "\n  "
        
        for x in range(len(timeserie_weirs)):  
            c = ET.SubElement(myroot[4],gn_dimr_brackets+"item")
            c.text = "\n      "
            c.tail = "\n  " 

            d = ET.SubElement(c,gn_dimr_brackets+"sourceName")
            d.text = "[Output]"+str(timeserie_weirs[x])+"/Crest level (s)"
            d.tail = "\n      "

            e = ET.SubElement(c,gn_dimr_brackets+"targetName")
            e.text = "weirs/"+str(timeserie_weirs[x])+"/CrestLevel"
            e.tail = "\n  "
    
    
    f = ET.SubElement(myroot[4],gn_dimr_brackets+"logger")
    f.text = "\n      "
    f.tail = "\n  " 
    
    g = ET.SubElement(f,gn_dimr_brackets+"workingDir")
    g.text = "."
    g.tail = "\n      "
    
    h = ET.SubElement(f,gn_dimr_brackets+"outputFile")
    h.text = "rtc_to_flowfm.nc"
    h.tail = "\n  "    
    
    if pidrule ==1:
        for x in range(len(weirs_auto)):
            c = ET.SubElement(myroot[5],gn_dimr_brackets+"item")
            c.text = "\n      "
            c.tail = "\n  " 

            d = ET.SubElement(c,gn_dimr_brackets+"sourceName")
            d.text = "observations/"+str(observationpoints[x])+"/water_level"
            d.tail = "\n      "

            e = ET.SubElement(c,gn_dimr_brackets+"targetName")
            e.text = "[Input]"+str(observationpoints[x])+"/Water level (op)"
            e.tail = "\n  "

    if intervalrule ==1:
        for x in range(len(interval_pumps)):
            c = ET.SubElement(myroot[5],gn_dimr_brackets+"item")
            c.text = "\n      "
            c.tail = "\n  " 

            d = ET.SubElement(c,gn_dimr_brackets+"sourceName")
            d.text = "observations/"+str(observationpoints_int_pumps[x])+"/water_level"
            d.tail = "\n      "

            e = ET.SubElement(c,gn_dimr_brackets+"targetName")
            e.text = "[Input]"+str(observationpoints_int_pumps[x])+"/Water level (op)"
            e.tail = "\n  "
        
        for x in range(len(interval_weirs)):
            c = ET.SubElement(myroot[5],gn_dimr_brackets+"item")
            c.text = "\n      "
            c.tail = "\n  " 

            d = ET.SubElement(c,gn_dimr_brackets+"sourceName")
            d.text = "observations/"+str(observationpoints_int[x])+"/water_level"
            d.tail = "\n      "

            e = ET.SubElement(c,gn_dimr_brackets+"targetName")
            e.text = "[Input]"+str(observationpoints_int[x])+"/Water level (op)"
            e.tail = "\n  "

    f = ET.SubElement(myroot[5],gn_dimr_brackets+"logger")
    f.text = "\n      "
    f.tail = "\n  " 
    
    g = ET.SubElement(f,gn_dimr_brackets+"workingDir")
    g.text = "."
    g.tail = "\n      "
    
    h = ET.SubElement(f,gn_dimr_brackets+"outputFile")
    h.text = "flowfm_to_rtc.nc"
    h.tail = "\n  "   

    #writing the root to a new xml file
    xmlpath = os.path.join(outputdir,"dimr_config.xml")
    print("RCT DIMR Config xml will be written to " + xmlpath)
    RTCdimr_config.write(xmlpath)

    xml = (bytes('<?xml version="1.0" encoding="utf-8" standalone="yes"?>\n', encoding='utf-8') + ET.tostring(myroot))
    xml = xml.decode('utf-8')
    with open(xmlpath, 'w+') as f:
        f.write(xml)

RCT DIMR Config xml will be written to D:\GITHUB\PILOT_HENA\modelbouwscript\dellen\dimr_config.xml


In [84]:
if RTC == 1: 
    # write bat file to run the dimr (please copy the path of the run_dimr file in the deltashell installation folder)
    batfile = """@ echo off 
    call "c:\Program Files\Deltares\D-HYDRO Suite 1D2D (1.0.0.53506)\plugins\DeltaShell.Dimr\kernels\x64\dimr\scripts\run_dimr.bat" dimr_config.xml -d 0 > dimr.log 2>&1 
    pause
    :end 
    """
    with open(os.path.join(outputdir,"dimr_bat.bat"), 'w+') as f:
        f.write(batfile)

In [85]:
#dfmmodel.mdu_parameters["refdate"] = 20000101
#dfmmodel.mdu_parameters["tstart"] = 0.0 * 3600
#dfmmodel.mdu_parameters["tstop"] = 144.0 * 1 * 3600
#dfmmodel.mdu_parameters["hisinterval"] = "120. 0. 0."
#dfmmodel.mdu_parameters["cflmax"] = 0.7


#dimr_path = r"dummypath\run_dimr.bat"
#dfmmodel.dimr_path = dimr_path
#fm_writer = DFlowFMWriter(dfmmodel, output_dir="dellen", name="dellen")

#fm_writer.objects_to_ldb()
#fm_writer.write_all()

if Path(r".\dellen\dflowfm\dellen.mdu").exists():
    print("Model is weggeschreven")
else:
    print("Er is geen model geschreven. Waarschijnlijk is iets fout gegaan")

Model is weggeschreven


## Importeren in de D-Hydro suite

Open het model nu in D-Hydro:
1. Open een "Empty Project"
2. In de "Home" ribbon, ga naar "Import" en selecteer "Flow Flexible Mesh Model"
3. Selecteer het bestand "boezemmodel.mdu" in ".\dfm_model\fm\boezemmodel.mdu"
4. Wacht tot het model is geimporteerd.....

Het geimporteerde model is nu zichtbaar in de D-Hydro suite
<img src="png/boezemmodel.png"/>

In [86]:
#print(dfmmodel.external_forcings.laterals.keys())
#print(drrmodel.external_forcings.boundary_nodes.keys())